[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/DWT/blob/master/docs/YCoCg_2D_DWT.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/DWT/blob/master/docs/YCoCg_2D_DWT.ipynb)

# Image Compression with YCoCg/DWT + 2D-DWT

## Parameters

In [1]:
import os
import math

In [2]:
try:
    from skimage import io
except:
    !pip install scikit-image
    from skimage import io

In [3]:
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np    

In [4]:
try:
    import matplotlib
    import matplotlib.pyplot as plt
    import pylab
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import pylab
%matplotlib inline

In [5]:
try:
    import pywt
except:
    !pip install pywavelets
    import pywt    

In [6]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

!ln -sf ~/repos/image_IO/logging_config.py .
!ln -sf ~/repos/image_IO/image_3.py .
import image_3 as RGB_image
!ln -sf ~/repos/image_IO/image_1.py .
import image_1 as gray_image
!ln -sf ~/repos/DWT/color_dyadic_DWT.py .

In [24]:
#import color_dyadic_DWT as DWT
try:
    #from DWT.color_dyadic_DWT import analyze as DWT
    #import DWT
    from DWT import color_dyadic_DWT as DWT
except:
    !pip install "DWT @ git+https://github.com/vicente-gonzalez-ruiz/DWT"
    from DWT import color_dyadic_DWT as DWT
    #import DWT
    #from DWT.color_dyadic_DWT import analyze as DWT

!ln -sf ~/information_theory/information.py .
import information
!ln -sf ~/information_theory/distortion.py .
import distortion

In [8]:
try:
    from information_theory.information import entropy as compute_entropy
    from information_theory.information import energy as compute_energy                         
    from information_theory.distortion import RMSE
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    from information_theory.information import entropy as compute_entropy
    from information_theory.information import energy as compute_energy                         
    from information_theory.distortion import RMSE

#!ln -sf ~/repos/YCoCg/YCoCg.py .
#import YCoCg as YUV
!ln -sf ~/repos/DCT/color_DCT.py .
import color_DCT as YUV

In [20]:
try:
    #from color_transforms.YCoCg import from_RGB
    #from color_transforms.YCoCg import to_RGB
    #from color_transforms.YCoCg import name as YUV_name
    from color_transforms import YCoCg as YUV
except:
    !pip install "color_transforms @ git+https://github.com/vicente-gonzalez-ruiz/color_transforms"
    #from color_transforms.YCoCg import from_RGB
    #from color_transforms.YCoCg import to_RGB
    #from color_transforms.YCoCg import name as YUV_name
    from color_transforms import YCoCg as YUV

!ln -sf ~/repos/scalar_quantization/quantization.py .
import quantization
!ln -sf ~/repos/scalar_quantization/deadzone_quantization.py .
#import deadzone_quantization as deadzone
from deadzone_quantization import Deadzone_Quantizer as Quantizer

In [10]:
try:
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer                          
except:
    !pip install "scalar_quantization @ git+https://github.com/vicente-gonzalez-ruiz/scalar_quantization"
    from scalar_quantization.deadzone_quantization import Deadzone_Quantizer as Quantizer

## Configuration

HOME = os.environ["HOME"]
#test_image = "../sequences/stockholm/"
test_image = HOME + "/repos/MRVC/images/lena_color/"
#test_image = HOME + "/repos/MRVC/images/white/"
#test_image = "../images/lena_bw/"
#test_image = HOME + "/repos/MRVC/images/circle/"

#quantizer = deadzone.Deadzone_Quantizer

#RGB_image.write = RGB_image.debug_write # Faster, but lower compression
#RGB_image.write = information.write # The fastest, but returns only an estimation of the length
#gray_image.write = gray_image.debug_write # Faster, but lower compression
#gray_image.write = information.write # The fastest, but returns only an estimation of the length

In [11]:
fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"

In [12]:
#wavelet_name = "Haar"
#wavelet_name = "db3"
wavelet_name = "db5"
#wavelet_name = "db7"
#wavelet_name = "db15"
#wavelet_name = "bior3.1"
#wavelet_name = "bior3.3"
#wavelet_name = "bior5.5"
wavelet = pywt.Wavelet(wavelet_name)
print(wavelet)
N_levels = 5
N_components = 3

Wavelet db5
  Family name:    Daubechies
  Short name:     db
  Filters length: 10
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False


## Subband-components information
Information about the coefficients after using a color transform and the DWT.

In [14]:
def info(sbc):
    max = sbc.max()
    min = sbc.min()
    max_min = max - min
    if max_min > 0:
        log2_max_min = math.ceil(math.log(max_min)/math.log(2))
    else:
        log2_max_min = 0
    avg = np.average(sbc)
    dev = math.sqrt(np.var(sbc))
    entropy = compute_entropy(sbc.flatten().astype(np.int16))
    energy = compute_energy(sbc)
    avg_energy = compute_energy(sbc)/sbc.size
    shape = sbc.shape
    return max, min, max_min, log2_max_min, avg, dev, entropy, energy, avg_energy, shape

def subbands_info(decomp):
    print("sorting subband-components by entropy")
    print("sbc maximum mininum    max-min average std-dev entropy        energy  avg-enegy shape")
    list_of_subbands_components = []
    sbc_index = 0
    accumulated_entropy = 0
    max_val = 0
    min_val = 10E10
    for c in range(N_components):
        sbc = decomp[0][..., c]
        infos = info(sbc)
        list_of_subbands_components.append((sbc_index, *infos))
        entropy = infos[5]
        accumulated_entropy += (entropy * sbc.size)
        if max_val < infos[0]:
            max_val = infos[0]
        if min_val > infos[1]:
            min_val = infos[1]
        sbc_index += 1
    for sr in decomp[1:]:
        for sb in sr:
            for c in range(N_components):
                sbc = sb[..., c]
                infos = info(sbc)
                list_of_subbands_components.append((sbc_index, *infos))
                entropy = infos[5]
                accumulated_entropy += (entropy * sbc.size)
                if max_val < infos[0]:
                    max_val = infos[0]
                if min_val > infos[1]:
                    min_val = infos[1]
                sbc_index += 1
    sorted_list_of_subbands_components = sorted(list_of_subbands_components, key=lambda x: x[6])[::-1]
    for _i in sorted_list_of_subbands_components:
        sbc_index, max, min, max_min, log2_max_min, avg, dev, entropy, energy, avg_energy, shape = _i
        print(f"{sbc_index:3d} {max:7.1f} {min:7.1f} {max_min:7.1f} {log2_max_min:>2d} {avg:7.1f} {dev:7.1f} {entropy:7.1f} {energy:13.1f} {avg_energy:10.1f} {shape}")

    avg_entropy = accumulated_entropy / img.size
    print("Image path:", fn)
    print("Wavelet name:", wavelet)
    print("Number of levels:", N_levels)
    print("Number of subbands:", int((sbc_index+1)/3))
    print("Number of subband-components:", sbc_index+1)
    print("Average entropy in the wavelet domain:", avg_entropy)
    print("Entropy in the image domain:", compute_entropy(img.flatten().astype(np.uint8)))
    print("Maximum coefficient value:", max_val)
    print("Minimum coefficient value:", min_val)
    print("Dynamic range in the transform domain:", max_val - min_val)
    print("Number of bits required for encoding the coefficients using integer numbers:", math.ceil(math.log(max_val - min_val)/math.log(2)))

In [15]:
#N_components = 3
#wavelet_name = "Haar"
#wavelet_name = "db3"
_wavelet_name = "db5"
#wavelet_name = "db7"
#wavelet_name = "db15"
#wavelet_name = "bior3.1"
#wavelet_name = "bior3.3"
#wavelet_name = "bior5.5"
_wavelet = pywt.Wavelet(wavelet_name)
print(_wavelet)
_N_levels = 1

#img = RGB_image.read(test_image).astype(np.int16)
img = io.imread(fn).astype(np.int16)
#YUV_img = YUV.from_RGB(img)
YUV_img = from_RGB(img)
Y = YUV_img[..., 0]
Y_avg = np.average(Y)
print(f"luma avg={Y_avg}")
Y -= int(Y_avg)
decom = pywt.wavedec2(data=Y, wavelet=_wavelet, level=_N_levels)
subbands_info(decom)

Wavelet db5
  Family name:    Daubechies
  Short name:     db
  Filters length: 10
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False
luma avg=120.56250381469727
sorting subband-components by entropy
sbc maximum mininum    max-min average std-dev entropy        energy  avg-enegy shape
  2    87.3  -174.6   261.9  9   -53.7    72.2     6.8     2105419.5     8097.8 (260,)
  1    90.5  -175.0   265.6  9   -53.1    70.4     6.8     2021365.3     7774.5 (260,)
  0    88.2  -175.8   264.1  9   -52.0    68.4     6.9     1917835.6     7376.3 (260,)
  8     8.6   -28.2    36.7  6    -0.1     3.4     3.4        3060.1       11.8 (260,)
  4     8.5   -11.0    19.5  5     0.1     3.0     3.4        2413.9        9.3 (260,)
  3    10.0   -10.8    20.8  5    -0.2     3.0     3.2        2333.5        9.0 (260,)
 11     7.0    -7.4    14.4  4    -0.0     2.6     3.1        1735.8        6.7 (260,)
  5     6.8    -8.6    15.4  4    

### Comments
1. Most of the energy (and entropy, i.e., information) is concentrated in the low-frequency subbands.
2. The wavelet domain is potentially more compressible than the image domain, because the entropy is smaller.
3. The number of bits required for representing the low-frequency subbands is significantly higher than the original 8-bits/component. This number depends on the wavelet filters, the number of levels of the transform, and the original image.

## "Lossless" compression
Notice that, since the color and spatial transforms use ploating-point arithmetic, it is impossible to guarantee the full reversibility of the encoding system.

### Using the "glued" representation
In the glued mode, all the coefficients are written in a single entropy coded file. Notice that we are using 16 bits/coefficient.

In [25]:
def lossless_glued(img, YUV, wavelet, N_levels):
    if YUV.name == "YCoCg":
        img = img.astype(np.int16)
    YUV_img = YUV.from_RGB(img)
    avgs = [np.average(YUV_img[...,c]) for c in range(3)]
    logger.info(f"avgs={avgs}")
    for c in range(3):
        YUV_img[..., c] -= int(avgs[c])
    decom = DWT.analyze(YUV_img, wavelet, N_levels)
    decom_ = DWT.add(decom, 32768)
    decom__ = DWT.set_type(decom_, np.uint16)
    output_len, slices = DWT.write_glued(decom__, "/tmp/lossless", 0)
    
    # Now we decompress the image to compute the RMSE
    _decom = DWT.read_glued(slices, "/tmp/lossless", 0)
    _decom2 = DWT.add(_decom, -32768)
    _YUV_img = DWT.synthesize(_decom2, wavelet, N_levels)
    for c in range(3):
        _YUV_img[..., c] += int(avgs[c])
    _img = (YUV.to_RGB(_YUV_img)).astype(np.uint8)
    _RMSE = distortion.RMSE(img, _img)
    return output_len, _RMSE

In [26]:
print(lossless_glued(img, YUV, wavelet, N_levels))

AttributeError: module 'DWT.color_dyadic_DWT' has no attribute 'add'

### Using the "unglued" representation
In this case, depending on the subband, we will use 8 or 16 bits/coefficient in the subband-components. Notice that "empty" subbands are not encoded (although without quantization, this is quite unlikely).

In [ ]:
def drange(x):
    return x.max() - x.min()

def add_offset(x):
    if drange(x) < 256:
        x = (x + 128).astype(np.uint8)
    else:
        x = (x + 32768).astype(np.uint16)
    return x

def sub_offset(x):
    if drange(x) < 256:
        x = x.astype(np.float64) - 128
    else:
        x = x.astype(np.float64) - 32768
    return x

In [ ]:
def lossless_unglued(img, YUV, wavelet, N_levels):
    if YUV.name == "YCoCg":
        img = img.astype(np.int16)
    YUV_img = YUV.from_RGB(img)
    avgs = [np.average(YUV_img[...,c]) for c in range(3)]
    logger.info(f"avgs={avgs}")
    for c in range(3):
        YUV_img[..., c] -= int(avgs[c])
    decom = DWT.analyze(YUV_img, wavelet, N_levels)
    always_positive_decom = [add_offset(decom[0])] # always_positive_decom is PNG friendly
    for sr in decom[1:]: # sr = spatial_resolution
        always_positive_sr = []
        for sb in sr: # sb = subband
            always_positive_sr.append(add_offset(sb))
        always_positive_decom.append(tuple(always_positive_sr))
    output_len, slices = DWT.write_unglued(always_positive_decom, "/tmp/lossless", 0)

    # Now we decompress the image to compute the RMSE
    _decom = [sub_offset(always_positive_decom[0])]
    for always_positive_sr in always_positive_decom[1:]:
        sr = []
        for always_positive_sb in always_positive_sr:
            sr.append(sub_offset(always_positive_sb))
        _decom.append(tuple(sr))
    _YUV_img = DWT.synthesize(_decom, wavelet, N_levels)
    for c in range(3):
        _YUV_img[..., c] += int(avgs[c])
    _img = YUV.to_RGB(_YUV_img).astype(np.uint8)
    _RMSE = distortion.RMSE(img, _img)
    return output_len, _RMSE

print(lossless_unglued(img, YUV, wavelet, N_levels))

The "unglued" representation is more efficient. This is probably due to the "reset" of context that occurs when we compress each subband in a different codestream.

Notice also that, in the case of "lena", JPEG2000 outputs 446411 bytes, and it is true lossless. This basically means that the entropy encoding algorithm used in JPEG2000 compress more than PNG. This probably happens because we are not exploiting the correlation between subbands.

## Visualization of the DWT (RGB) domain

In [ ]:
decom = DWT.analyze(img, wavelet, 3)
glued_decom, shapes = DWT.glue_color_decomposition(decom)
RGB_image.show_normalized(glued_decom, "Glued decomposition")

In [ ]:
def normalize(sb):
    return (sb - sb.min()) / (sb.max() - sb.min())

decom = DWT.analyze(img, wavelet, 3)
view_LL = normalize(decom[0])
view_decom = [view_LL]
RGB_image.show(view_LL, '')
for sr in decom[1:]:
    view_sr = []
    for sb in sr:
        view_sb = normalize(sb)
        view_sr.append(view_sb)
        RGB_image.show(view_sb, '')
    view_decom.append(tuple(view_sr))
glued_decom, shapes = DWT.glue_color_decomposition(view_decom)
RGB_image.show(glued_decom, "Glued decomposition")

## RD performance

All subbands are quantized with the same quantization step. Notice that all subbands should have the same gain, and obviously, the transforms must be orthogonal. The distortion is measured in the image domain.

### Subband gains

Not all wavelet filters have the same gain in all the subbands. The gain $G_b$ of a subband $b$ can be computed as the squared norm of the DWT synthesis basis vector of the corresponding subband (see Page 438, Chapter 10, JPEG2000 Image Compression Fundamentals, Standards and Practice). For the subband $b$, such quantity is the energy (sum of squared sample values) in an image reconstructed from exactly one unit amplitude sample in $b$ (i.e., setting all other samples in $b$ and all samples in all other subbands to 0).

$G_b$ is identical for all samples in subband $b$ which are located sufficiently far from the image boundaries to not be affected by any symmetric extension procedure.

In [ ]:
# The "img_shape" parameter should be large enough to avoid
# any interference caused by the extension procedure.
def compute_normalized_gains(img_shape, wavelet, N_levels):
    gains = []
    x = np.zeros(shape=img_shape)
    decom = DWT.analyze(x, wavelet, N_levels)
    decom[0][decom[0].shape[0]//2, decom[0].shape[1]//2] = 1
    #decom[0][...] = 1
    z = DWT.synthesize(decom, wavelet, N_levels)
    gains.append(information.energy(z))
    prev_sb = decom[0]
    for sr in decom[1:]:
        for sb in sr:
            prev_sb[...] = 0
            #sb[...] = 1#coeff_value/sb.size
            sb[sb.shape[0]//2, sb.shape[1]//2] = 1
            z = DWT.synthesize(decom, wavelet, N_levels)
            gains.append(information.energy(z))
            prev_sb = sb
    
    return gains/sum(gains)

_wavelet_name = "db5"
#_wavelet_name = "bior3.5"
#_wavelet_name = "bior5.5"
_wavelet = pywt.Wavelet(_wavelet_name)
gains = compute_normalized_gains(img.shape, _wavelet, N_levels)
print(gains)

### Quantization step sizes

The gain $G_b$ of a subband $b$ should be considered in the quantization process because to minimize the quantization error (in terms of the MSE), those coefficients with a higher gain (that on average are going to have a larger dynamic range) should be quantized proportionally using
\begin{equation}
  \Delta_b = \Delta\sqrt{\frac{G_0}{G_b}}
\end{equation}
where $\Delta$ is a "base step size", $G_b$ is the squared norm of the synthesis filter for the subband $b$, and $\Delta_b$ is que quantization step size that should be applied to the subband $b$ (see Eq. (10.23), Chapter 10, JPEG2000 Image Compression Fundamentals, Standards and Practice). $G_0$ should be the subband gain of the LL subband.Ahora

In [ ]:
gains = compute_normalized_gains(img.shape, wavelet, N_levels)
print(gains)

In [ ]:
#Q_factors = [1/math.sqrt(i) for i in gains]
#print(Q_factors)
#Q_factors = [math.sqrt(i/gains[0]) for i in gains]
#print(Q_factors)

In [ ]:
# Use uniform quantization and write_unglued()
#Q_steps = [32, 16, 8, 4, 2, 1, 1/2, 1/4, 1/8, 1/16, 1/32]
#c = 0
#for s in Q_steps:
#    Q_steps[c] *= 64
#    c += 1
Q_steps = [256, 128, 64, 32, 16, 8]
#Q_steps = [512, 256, 128, 64, 32, 16]

In [ ]:
def get_BPP(img, decom, Q_step, Q_factors, wavelet, N_levels, write):
    Q = Quantizer(Q_step=Q_step*Q_factors[0])
    LL = decom[0]
    LL_k = Q.quantize(LL)
    LL_dQ = Q.dequantize(LL_k)
    decom_k = [add_offset(LL_k)]
    decom_dQ = [LL_dQ]
    sbc_counter = 1
    print(Q_step, Q_factors[0])
    print("sb_Q_factor =", Q_step*Q_factors[0], end=' ')
    for sr in decom[1:]:
        sr_k = []
        sr_dQ = []
        for sb in sr: # sb = subband
            sb_Q_factor = Q_step*Q_factors[sbc_counter]
            print(sb_Q_factor, end=' ')
            Q = Quantizer(Q_step=sb_Q_factor)
            sb_k = Q.quantize(sb)
            sb_dQ = Q.dequantize(sb_k)
            sr_k.append(add_offset(sb_k))
            sr_dQ.append(sb_dQ)
            sbc_counter += 1
        decom_k.append(tuple(sr_k))
        decom_dQ.append(tuple(sr_dQ))
    print()
    BPP = (write(decom_k, f"/tmp/{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    return BPP, decom_dQ

In [ ]:
def compute_RD_curve(img, YUV, wavelet, N_levels, Q_factors, Q_steps, write):
    print("Q_steps =", Q_steps)
    print("Q_factors =", Q_factors)
    YUV_img = YUV.from_RGB(img)
    avgs = [np.average(YUV_img[..., c]) for c in range(3)]
    print(f"avgs={avgs}")
    for c in range(3):
        YUV_img[..., c] -= int(avgs[c])
    decom = DWT.analyze(YUV_img, wavelet, N_levels)
    RD_curve = []
    for Q_step in Q_steps:
        BPP, decom_dQ = get_BPP(img, decom, Q_step, Q_factors, wavelet, N_levels, write)
        YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
        for c in range(3):
            YUV_img_dQ[..., c] += int(avgs[c])
        img_dQ = YUV.to_RGB(YUV_img_dQ)
        img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
        RMSE = distortion.RMSE(img, img_dQ)
        print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
        #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")
        RD_curve.append((BPP, RMSE))
    return RD_curve

### Using the glued representation

In [ ]:
Q_factors = [1]*(N_levels*3+1)
img = RGB_image.read(test_image).astype(np.int16)
DWT_constantQ_glued = compute_RD_curve(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_glued)

In [ ]:
Q_factors = [math.sqrt(gains[0]/i) for i in gains]
img = RGB_image.read(test_image).astype(np.int16)
DWT_factorsQ_glued = compute_RD_curve(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_glued)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_constantQ_glued), label=f"Constant Q_step")
pylab.plot(*zip(*DWT_factorsQ_glued), label=f"Using factors")
#pylab.plot(*zip(*image_domain_DWT_RD_points2), label=f"DWT uniform Q (image domain)")
#pylab.plot(*zip(*transform_domain_DWT_RD_points), label=f"DWT uniform Q (transform domain)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

In the case of using biorthogonal transforms, the use of the subband gains improves slighly the RD performance.

### Using the unglued representation

In [ ]:
Q_factors = [1]*(N_levels*3+1)
img = RGB_image.read(test_image).astype(np.int16)
DWT_constantQ_unglued = compute_RD_curve(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_unglued)

In [ ]:
Q_factors = [math.sqrt(gains[0]/i) for i in gains]
img = RGB_image.read(test_image).astype(np.int16)
DWT_factorsQ_unglued = compute_RD_curve(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_unglued)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_constantQ_unglued), label=f"Constant Q_step")
pylab.plot(*zip(*DWT_factorsQ_unglued), label=f"Using factors")
#pylab.plot(*zip(*image_domain_DWT_RD_points2), label=f"DWT uniform Q (image domain)")
#pylab.plot(*zip(*transform_domain_DWT_RD_points), label=f"DWT uniform Q (transform domain)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

Again, if we use biorthogonal transforms, the use of the subband gains improves slighly the RD performance.

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_factorsQ_glued), label=f"glued")
pylab.plot(*zip(*DWT_factorsQ_unglued), label=f"unglued")
#pylab.plot(*zip(*image_domain_DWT_RD_points2), label=f"DWT uniform Q (image domain)")
#pylab.plot(*zip(*transform_domain_DWT_RD_points), label=f"DWT uniform Q (transform domain)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

Writting each subband in a separate file (unglued) is more efficient than generating a single file (glued).

## Using only 8 bits/coefficient
The kernels used in the DWT expand the dynamic range of the image (the coefficients need more bits to be represented). One way to ensure that we will need only 8 bits/coefficient (probably increasing the R performance of the entropy codec) is to "pre-quantize" the coefficients, using, for example, the following quantization pattern (example for 3 levels):

    +---+---+-------+---------------+
    | 8 | 4 |       |               |
    +---+---+   2   |               |
    | 4 | 4 |       |               |
    +---+---+-------+       1       |
    |       |       |               |
    |   2   |   2   |               |
    |       |       |               |
    +-------+-------+---------------+
    |               |               |
    |               |               |
    |               |               |
    |       1       |       1       |
    |               |               |
    |               |               |
    |               |               |
    +---------------+---------------+

In [ ]:
#Q_steps = [256, 128, 64, 32, 16, 8]
#Q_steps = range(256, 16, -4)

#### Testing the 8<->16 bits/coefficient converters

In [ ]:
def to_8bits(decomposition): # compact?
    '''Remove the least significant bit-planes of the <decomposition> to represent each coefficient with 8 bits.
    
    Parameters
    ----------
    decomposition: list
        The decomposition to quantize.
        
    Returns:
    decomposition: list
        The quantized decomposition.
    
    '''
    N_levels = len(decomposition) - 1
    #new_decomp = [(decomposition[0].astype(np.int16) >> N_levels).astype(np.uint8)]
    new_decomp = [(decomposition[0].astype(np.int16) >> N_levels)]
    levels_counter = N_levels - 1
    for resolution in decomposition[1:]:
        new_resol = []
        for subband in resolution:
            #new_resol.append((subband.astype(np.int16) >> levels_counter).astype(np.uint8))
            new_resol.append((subband.astype(np.int16) >> levels_counter))
        new_decomp.append(tuple(new_resol))
        levels_counter -= 1
    #print("to_8bits:", new_decomp[0].dtype)
    return new_decomp

def to_16bits(decomposition): # uncompact ?
    N_levels = len(decomposition) - 1
    new_decomp = [decomposition[0].astype(np.int16) << N_levels]
    levels_counter = N_levels - 1
    for resolution in decomposition[1:]:
        new_resol = []
        for subband in resolution:
            new_resol.append(subband.astype(np.int16) << levels_counter)
        new_decomp.append(tuple(new_resol))
        levels_counter -= 1
    return new_decomp

img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[...,c]) for c in range(3)]
#print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
#decom_8bit = to_8bits(decom)
decom = to_8bits(decom)
#print(decom_8bit[0].dtype)
#print(decom[0].dtype)
#glued_decom_8bit, shapes = DWT.glue_color_decomposition(decom_8bit)
glued_decom, shapes = DWT.glue_color_decomposition(decom)
#image_3.show(image_3.normalize(glued_decom_8bit))
#image_3.show((glued_decom_8bit + 128).astype(np.uint8), "128-shifted glued decomposition")
RGB_image.show((glued_decom + 128).astype(np.uint8), "glued decomposition")

In [ ]:
#decom_dQ = to_16bits(decom_8bit)
decom = to_16bits(decom)
#YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
YUV_img_dQ = DWT.synthesize(decom, wavelet, N_levels)
for c in range(3):
    YUV_img_dQ[..., c] += int(avgs[c])
img_dQ = (YUV.to_RGB(YUV_img_dQ)).astype(np.uint8)
RGB_image.show(img_dQ, "8-bit quantized and dequantized")

In [ ]:
e = img.astype(np.int16) - img_dQ
print(e.dtype, e.max(), e.min())
plt.hist(e[...,0].ravel(), bins=256)
plt.xlabel("Error value")
plt.ylabel("Ocurrences")
plt.show()

In [ ]:
RGB_image.show(RGB_image.normalize(e), "Normalized error")
RGB_image.show((e + 128).astype(np.uint8), "error")

In [ ]:
subbands_info(to_8bits(decom))

## RD performance using at most 8-bits/coefficient

### Using the glued representation

In [ ]:
def compute_Q_step(Q_step, level):
    #step = int(math.ceil(Q_step * math.pow(2, N_levels)))
    step_for_8b = int(math.ceil(Q_step / math.pow(2, level)))
    print(f"compute_Q_step: Q_step={Q_step} level={level + 1} step_for_8b={step_for_8b}")
    return step_for_8b

img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
DWT_8b_constantQ_glued = []
for Q_step in Q_steps:
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    Q = Quantizer(Q_step=_Q_step)
    LL_8b_k = Q.quantize(LL_8b) # Baybe bettter Q.get_indexes()
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k) # Q.get_signal()
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 0
    level = N_levels - 1
    for sr_8b in decom_8b[1:]: # sr = spatial_resolution
        sr_8b_k = []
        sr_8b_dQ = []
        for sb_8b in sr_8b: # sb = subband
            _Q_step = compute_Q_step(Q_step, level)
            Q = Quantizer(Q_step=_Q_step)
            sb_8b_k = Q.quantize(sb_8b)
            sb_8b_dQ = Q.dequantize(sb_8b_k)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    BPP = (DWT.write_glued(decom_8b_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    decom_dQ = to_16bits(decom_8b_dQ)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    DWT_8b_constantQ_glued.append((BPP, RMSE))
    #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

### Using the unglued representation

img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
DWT_8b_constantQ_unglued = []
for Q_step in Q_steps:
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    Q = Quantizer(Q_step=_Q_step)
    LL_8b_k = Q.quantize(LL_8b) # Baybe bettter Q.get_indexes()
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k) # Q.get_signal()
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 0
    level = N_levels - 1
    for sr_8b in decom_8b[1:]: # sr = spatial_resolution
        sr_8b_k = []
        sr_8b_dQ = []
        for sb_8b in sr_8b: # sb = subband
            _Q_step = compute_Q_step(Q_step, level)
            Q = Quantizer(Q_step=_Q_step)
            sb_8b_k = Q.quantize(sb_8b)
            sb_8b_dQ = Q.dequantize(sb_8b_k)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    BPP = (DWT.write_unglued(decom_8b_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    decom_dQ = to_16bits(decom_8b_dQ)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    DWT_8b_constantQ_unglued.append((BPP, RMSE))
    #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

In [ ]:
def get_RD_point_8b(img, decom_8b, Q_step, Q_factors, wavelet, N_levels, write):
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    Q = Quantizer(Q_step=_Q_step*Q_factors[0])
    LL_8b_k = Q.quantize(LL_8b)
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k)
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 1
    print(Q_step, Q_factors[0])
    print("sb_Q_factor =", Q_step*Q_factors[0], end=' ')
    level = N_levels - 1
    for sr_8b in decom_8b[1:]:
        sr_8b_k = []
        sr_8b_dQ = []
        for sb_8b in sr_8b:
            _Q_step = compute_Q_step(Q_step, level)
            sb_Q_factor = _Q_step*Q_factors[sbc_counter]
            print(sb_Q_factor, end=' ')
            Q = Quantizer(Q_step=sb_Q_factor)
            sb_8b_k = Q.quantize(sb_8b)
            sb_8b_dQ = Q.dequantize(sb_8b_k)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    print()
    BPP = (write(decom_8b_k, f"/tmp/{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    decom_dQ = to_16bits(decom_8b_dQ)
    return BPP, decom_dQ

In [ ]:
def compute_RD_curve_8b(img, YUV, wavelet, N_levels, Q_factors, Q_steps, write):
    print("Q_steps =", Q_steps)
    print("Q_factors =", Q_factors)
    YUV_img = YUV.from_RGB(img)
    avgs = [np.average(YUV_img[..., c]) for c in range(3)]
    print(f"avgs={avgs}")
    for c in range(3):
        YUV_img[..., c] -= int(avgs[c])
    decom = DWT.analyze(YUV_img, wavelet, N_levels)
    RD_curve = []
    for Q_step in Q_steps:
        BPP, decom_dQ = get_RD_point_8b(img, decom, Q_step, Q_factors, wavelet, N_levels, write)
        YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
        for c in range(3):
            YUV_img_dQ[..., c] += int(avgs[c])
        img_dQ = YUV.to_RGB(YUV_img_dQ)
        img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
        RMSE = distortion.RMSE(img, img_dQ)
        print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
        #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")
        RD_curve.append((BPP, RMSE))
    return RD_curve

In [ ]:
gains = compute_normalized_gains(img.shape, wavelet, N_levels)
print(gains)

In [ ]:
#Q_steps = [256, 128, 64, 32, 16, 8]

In [ ]:
Q_factors = [1]*(N_levels*3+1)
img = RGB_image.read(test_image).astype(np.int16)
DWT_constantQ_unglued_8b = compute_RD_curve_8b(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_unglued)

In [ ]:
Q_factors = [math.sqrt(gains[0]/i) for i in gains]
img = RGB_image.read(test_image).astype(np.int16)
DWT_factorsQ_unglued_8b = compute_RD_curve_8b(img, YUV, wavelet, N_levels, Q_factors, Q_steps, DWT.write_unglued)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_constantQ_unglued_8b), label=f"Constant Q_step")
pylab.plot(*zip(*DWT_factorsQ_unglued_8b), label=f"Using factors")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

Again, unglued a bit better.

## 8 bits/coefficient VS 16 bits/coefficient

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_constantQ_unglued), label=f"DWT uniform Q (16 bits)")
pylab.plot(*zip(*DWT_constantQ_unglued_8b), label=f"DWT uniform Q (8 bits)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

Slightly better the 8 bits/coefficient version.

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_factorsQ_unglued), label=f"Using factors (16 bits)")
pylab.plot(*zip(*DWT_factorsQ_unglued_8b), label=f"Using factors (8 bits)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

### Compare

In [ ]:
DCT_RD_points = []
with open("../YCoCg_2D_DCT_SQ/YCoCg_2D_DCT_SQ.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        DCT_RD_points.append((float(rate), float(_distortion)))

pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_RD_points), label=r"Deadzone(2D-DCT($\mathbf{\Delta}^{\mathrm{Y}}_i = \mathbf{\Delta}^{\mathrm{Co}}_i = \mathbf{\Delta}^{\mathrm{Cg}}_i)$)+PNG")
pylab.plot(*zip(*DWT_8b_constantQ), label=f"DWT uniform Q (8 bits)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

In [ ]:
JPEG_RD_points = []
with open(HOME + "/repos/JPEG/JPEG.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        JPEG_RD_points.append((float(rate), float(_distortion)))

In [ ]:
JPEG2K_RD_points = []
with open(HOME + "/repos/JPEG2000/JPEG2000.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        JPEG2K_RD_points.append((float(rate), float(_distortion)))

In [ ]:
_2D_VQ_RD_points = []
with open("../spatial_color_VQ/VQ_2D_RGB_RD_points.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        _2D_VQ_RD_points.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_RD_points), label=r"Deadzone(2D-DCT($\mathbf{\Delta}^{\mathrm{Y}}_i = \mathbf{\Delta}^{\mathrm{Co}}_i = \mathbf{\Delta}^{\mathrm{Cg}}_i)$)+PNG")
pylab.plot(*zip(*JPEG_RD_points), label=r"JPEG")
pylab.plot(*zip(*JPEG2K_RD_points), label=r"JPEG 2000")
pylab.plot(*zip(*DWT_factorsQ_unglued), label=f"Deadzone(DWT({YUV.name}))+PNG ({wavelet_name})")
pylab.plot(*zip(*_2D_VQ_RD_points), label="VQ")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

## Improving R

JPEG2000 and our codec are quite similar, except in the entropy codec (JPEG2000 uses a bit-plane encoder based on a [MQ-codec](https://ieeexplore.ieee.org/document/4540263), and we are using PNG).

If we visualize the subbands, it's obvious that there is a dependency between the three subbands of the same spatial resolution level $l$, and also a dependency between such subbands and the subband that represents the spatial resolution level $l+1$ (considering that if $l=0$ we have the original resolution).



In [ ]:
RGB_image.show(img)
decom = DWT.analyze(img, wavelet, 3)
sb_index = 0
RGB_image.show((normalize(decom[0])), '')
print(sb_index, decom[0].max(), decom[0].min())
for sr in decom[1:]:
    view_sr = []
    for sb in sr:
        sb_index += 1
        view_sb = (normalize(sb) * 255).astype(np.uint8)
        view_sr.append(view_sb)
        RGB_image.show(view_sb, '')
        print(sb_index, sb.max(), sb.min())
    view_decom.append(tuple(view_sr))

Other interesting aspects that worth noting are:
1. Orthogonal transforms are not linear in phase, which implies that the coefficients that result from the same structure are placed in different coordinates in the transform domain, with respect to the coordinates of the coefficients in the previous resolution level. However, orthogonal transform are more efficient than biorthogonal transforms concentrating energy.
2. Inside of a filtering orientation (horizontal, vertical, o diagonal), the pattern shown by a 2D block in a subband of the level $l$ tends to be found in the level $l-1$.

### A proposal

The idea of the following encoder is exploit the local spatial correlation with VQ and the global one with PNG, and to exploit the multiresolution correlation reusing the code-books, as much as possible, inside of each spatial orientation. In the following algorithm, all the code-books have a number of code-vectors of size 2x2 (the code-book length controls R and obviously, D, and therefore, no scalar quantization is required). Notice that this algorithm should perform well with any dyadic DWT transform, regardless of its phase linearity.

1. Apply the pre-quantization pattern described previously to homogenize the dynamic range between spatial resolution levels.
2. VQ the subband LL$^L$ ($L$ is the number of levels of the DWT). Compress the code-book with GZIP and send it. Compress the indexes with PNG.
3. For each of the filtering orientations (LH, HL, and HH):
    1. VQ the subband (of the level $L$). GZIP the code-book and send it. Compress the indexes with PNG.
    2. For each of the rest of levels, starting with the highest one ($l=L-1$):
        1. VQ the subband (in the level $l$).
        2. Substract to the current code-book the used in the spatial resolution level $l+1$. GZIP the code-book and send it. Compress the indexes with PNG.

In [ ]:
with open('YCoCg_2D_DWT_SQ.txt', 'w') as f:
    for item in DWT_factorsQ_unglued_8b:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
import time
while True:
    time.sleep(1)

## Optimize the Q_steps of the subbands depending on their RD contribution

1. Select a Q_step for subband LL and compute the RD slope = reduction of the distortion (compared to the "gray" image) / number of bits required.
2. Select the same Q_step for the next subband and compute the RD slope = reduction of the distortion (compared to the previous reconstructed image) / number of bits required. If the slope i higher, decrease the Q_step. If the slope is lower, increase the Q_step.

In [ ]:
#Q_steps = range(256, 32, -4)

img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
optimized_Q_step = []
for Q_step in Q_steps:
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    Q = Quantizer(Q_step=_Q_step)
    LL_8b_k = Q.quantize(LL_8b) # Baybe bettter Q.get_indexes()
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k) # Q.get_signal()
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 0
    level = N_levels - 1
    for sr_8b in decom_8b[1:]: # sr = spatial_resolution
        sr_8b_k = []
        sr_8b_dQ = []
        for sb_8b in sr_8b: # sb = subband
            _Q_step = compute_Q_step(Q_step, level)
            Q = Quantizer(Q_step=_Q_step)
            #Q = Quantizer(Q_step=100000)
            sb_8b_k = Q.quantize(sb_8b)
            sb_8b_dQ = Q.dequantize(sb_8b_k)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            print((sb_8b_k + 128).min(), (sb_8b_k + 128).max())
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    BPP = (DWT.write_unglued(decom_8b_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    decom_dQ = to_16bits(decom_8b_dQ)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    optimized_Q_step.append((BPP, RMSE))
    #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*optimized_Q_step), label=f"optimized Q_step")
pylab.plot(*zip(*DWT_8b_constantQ_unglued), label=f"DWT uniform Q (8 bits)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc='upper right')
#plt.xlim([0, 1])
pylab.show()

In [ ]:
img = RGB_image.read(test_image).astype(np.int16)
YUV_img = YUV.from_RGB(img)
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)

gray_img = np.ones_like(img)*128
YUV_gray_img = YUV.from_RGB(gray_img)
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
gray_decom = DWT.analyze(YUV_gray_img, wavelet, N_levels)

optimized_Q_step = []
for Q_step in Q_steps: # "Master" Q_step used for the LL subband.
    # Slope LL subband
    Q = Quantizer(Q_step=Q_step)
    LL = decom[0]
    LL_k = Q.quantize(LL) # Baybe bettter Q.get_indexes()
    LL_dQ = Q.dequantize(LL_k) # Q.get_signal()
    decom_k = [add_offset(LL_k)]
    decom_dQ = [LL_dQ]
    sbc_counter = 0
    levels_counter = N_levels - 1
    Q = Quantizer(Q_step=10000) # Rest high-freq subbands to zero
    for sr in decom[1:]: # sr = spatial_resolution
        sr_k = []
        sr_dQ = []
        for sb in sr: # sb = subband
            sb_k = Q.quantize(sb)
            sb_dQ = Q.dequantize(sb_k)
            sr_k.append(add_offset(sb_k))
            sr_dQ.append(sb_dQ)
            sbc_counter += 1
        decom_k.append(tuple(sr_k))
        decom_dQ.append(tuple(sr_dQ))
        levels_counter += 1
    N_bytes = RGB_image.write(add_offset(LL_k), f"/tmp/1_", 0)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    optimized_Q_step.append((BPP, RMSE))
    #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

In [ ]:
# Borrame
img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
DWT_8b_constantQ_unglued2 = []
for Q_step in Q_steps:
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    Q = Quantizer(Q_step=_Q_step)
    LL_8b_k = Q.quantize(LL_8b) # Baybe bettter Q.get_indexes()
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k) # Q.get_signal()
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 0
    level = N_levels - 1
    energy_BPP_cost = []
    for sr_8b in decom_8b[1:]: # sr = spatial_resolution
        sr_8b_k = []
        sr_8b_dQ = []
        print("sbc_counter =", sbc_counter, "energy =", sbc_energy)
        BPP_cost = RGB_image.write((sb_8b_k + 128).astype(np.uint8), f"/tmp/1_", 0)
        print("BPP_cost =", BPP_cost)
        energy_BPP_cost.append(sbc_energy/BPP_cost)
        print("energy/BPP_cost =", energy__BPP_cost[-1])
        for sb_8b in sr_8b: # sb = subband
            _Q_step = compute_Q_step(Q_step, level)
            Q = Quantizer(Q_step=_Q_step)
            sb_8b_k = Q.quantize(sb_8b)
            sb_8b_dQ = Q.dequantize(sb_8b_k)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
            sbc_energy = information.energy(sb_8b_dQ)
            print("sbc_counter =", sbc_counter, "energy =", sbc_energy)
            BPP_cost = RGB_image.write((sb_8b_k + 128).astype(np.uint8), f"/tmp/1_", 0)
            print("BPP_cost =", BPP_cost)
            energy_BPP_cost.append(sbc_energy/BPP_cost)
            print("energy/BPP_cost =", energy__BPP_cost[-1])
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    
    BPP = (DWT.write_unglued(decom_8b_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/(YUV_img.shape[0]*YUV_img.shape[1])
    decom_dQ = to_16bits(decom_8b_dQ)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    DWT_8b_constantQ_unglued2.append((BPP, RMSE))
    #RGB_image.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

## Ignore the rest

In [ ]:
gains = []
x = np.zeros(shape=(512, 512, 3))
y = DWT.analyze(x, wavelet, N_levels)
coeff_value = y[0].size
y[0][...] = coeff_value/y[0].size
z = DWT.synthesize(y, wavelet, N_levels)
gains.append(information.average_energy(z))
prev_sb = y[0]
for sr in y[1:]:
    for sb in sr:
        prev_sb[...] = 0.0
        sb[...] = coeff_value/sb.size
        z = DWT.synthesize(y, wavelet, N_levels)
        gains.append(information.average_energy(z))
        prev_sb = sb
        
x = np.empty(shape=(512, 512, 3))
y = DWT.analyze(x, wavelet, N_levels)
coeff_value = y[0].size
y[0][...] = coeff_value/y[0].size
for sr in y[1:]:
    for sb in sr:
        sb[...] = coeff_value/sb.size
z = DWT.synthesize(y, wavelet, N_levels)
z_energy = information.average_energy(z)

gains = [gain/z_energy for gain in gains]
print("Unitary (normalized) inverse transform subband gains:", gains)
print(np.testing.assert_almost_equal(sum(gains), 1.0))
print(sum(gains))

### Measuring the distortion in the transform domain (TO-DO)

Only works if the colorand the spatial transforms are orthogonal!

In [ ]:
Q_steps = [256, 128, 64, 32, 16, 8, 4, 2, 1]
img = RGB_image.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
LL = decom[0]
for Q_step in Q_steps:
    Q = Quantizer(Q_step=Q_step)
    LL_k = Q.quantize(LL)
    LL_dQ = Q.dequantize(LL_k)
    # ....

## Uniform quantization using bi-orthogonal filters (TO-DO)

This procedure should be used when the YUV/DWT transform is not orthonormal (for example, when using the YCoCg color space and/or biorthogonal DWT filters). In this case we must consider the subband gains.

In [ ]:
print(wavelet)
#wavelet = 'bior3.5'
wavelet = 'db5'
img = image_3.read(test_image).astype(np.int16)
DWT.compute_gains(wavelet=wavelet, N_levels=N_levels, pixels_in_y=img.shape[0], pixels_in_x=img.shape[1])
# ...

## Finally ... can we increase the RD performance using RDO?

Let's do that, for a given RD point, the subband-components contribute (in general, approximately) the same to que quality of the reconstruction.

Algorithm:
1. Read the image.
2. Transform it to the YCoCg domain.
3. Transform each component to the DWT domain.
4. Estimate the RD curve for each subband-component.
5. Compute the slope of each step of each curve and put all the tuples (slopes, quantization steps, subband number) in the same list.
6. Sort the previous list by the slope field.
7. Compute the RD curve that progressively uses descending slopes.

### Read the image and move it to the 0-mean YCoCg domain.

In [ ]:
img = image_3.read(test_image, 0)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])

### Compute the 2D-DWT

In [ ]:
decom = DWT.analyze(YUV_img, wavelet, N_levels)

### Create a list of RD points for each RD curve (subband-component) and the corresponding lists of RD slopes between these points
There is a RD curve per subband-component. The first coordinate of each point is the rate and the second coordinate is the corresponding distortion.
For rate=0, the RMSE is the energy of the subband-component.
Notice that:
1. We have considered that the YUV/DWT transform is orthogonal and therefore, the distortion of the recontructed image can be estimated in the transform domain.
2. The slope cannot be computed for the first point of each RD curve.

In [ ]:
RD_points = []
RD_slopes = []
for _c in range(N_components):
    sbc = decom[0][..., _c]
    RD_points.append([(0, information.average_energy(sbc))]) # Work with RMSE's that are average distortions
    RD_slopes.append([])
level = N_levels - 1
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            sbc = sb[..., _c]
            sbc_avg_energy = information.average_energy(sbc)
            # The first point of each RD curve has a maximum distortion equal
            # to the energy of the subband and a rate = 0
            RD_points.append([(0, sbc_avg_energy)])
            RD_slopes.append([])
    level -= 1

In [ ]:
print("subband-component, RD-point")
for _i,_j in enumerate(RD_points):
    print(_i,_j)
    if not ((_i+1) % 3):
        print('')

#### Populate the rest of points of each curve

In [ ]:
# Subband LL
sbc_number = 0
for _c in range(N_components):
    sbc = decom[0][..., _c]
    Q_step_number = 0
    for Q_step in Q_steps:
        sbc_k = Q.quantize(sbc, Q_step)
        sbc_dQ = Q.dequantize(sbc_k, Q_step)
        RMSE = distortion.RMSE(sbc, sbc_dQ)
        #print(sbc.max(), sbc.min(), sbc_dQ.max(), sbc_dQ.min())
        #BPP = image_1.write((sbc_k + 128).astype(np.uint8), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
        BPP = image_1.write((sbc_k + 32768).astype(np.uint16), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
        #BPP = information.entropy(sbc_k.astype(np.int16).flatten())
        point = (BPP, RMSE)
        RD_points[sbc_number].append(point)
        delta_BPP = BPP - RD_points[_c][Q_step_number][0]
        delta_RMSE = RD_points[_c][Q_step_number][1] - RMSE
        if delta_BPP > 0:
            slope = delta_RMSE/delta_BPP
        else:
            slope = 0
        print(f"sbc_number={sbc_number} Q_step={Q_step} BPP={point[0]} RMSE={point[1]} slope={slope}")
        if slope > 0:
            RD_slopes[_c].append((Q_step, slope, _c))
        Q_step_number += 1
    sbc_number += 1

# Rest of subbands
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            sbc = sb[..., _c]
            Q_step_number = 0
            for Q_step in Q_steps:
                sbc_k = Q.quantize(sbc, Q_step)
                sbc_dQ = Q.dequantize(sbc_k, Q_step)
                RMSE = distortion.RMSE(sbc, sbc_dQ)
                #BPP = image_1.write((sbc_k + 128).astype(np.uint8), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
                BPP = image_1.write((sbc_k + 32768).astype(np.uint16), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
                #sbc_BPP = information.entropy(sbc_k.astype(np.int16).flatten())
                point = (BPP, RMSE)
                RD_points[sbc_number].append(point)
                delta_BPP = BPP - RD_points[sbc_number][Q_step_number][0]
                delta_RMSE = RD_points[sbc_number][Q_step_number][1] - RMSE
                if delta_BPP > 0:
                    slope = delta_RMSE/delta_BPP
                else:
                    slope = 0
                print(f"sbc_number={sbc_number} Q_step={Q_step} BPP={point[0]} RMSE={point[1]} slope={slope}")
                if slope > 0:
                    RD_slopes[sbc_number].append((Q_step, slope, sbc_number))
                Q_step_number += 1
            sbc_number += 1

In [ ]:
print("(BPP, RMSE)")
for _i, _j in enumerate(RD_points):
    print(_i, _j)
    if not ((_i+1) % 3):
        print('')

In [ ]:
print("subband-component, (Q_step, slope, subband-component number)")
for _i, _j in enumerate(RD_slopes):
    print(_i, _j)
    if not ((_i+1) % 3):
        print('')

## Remove points that do not belong to the convex-hull
Remove those points of each subband-component that do not satisfy that the slope is higher than the next point of the curve. 

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[1] < curr[1]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_slopes(i))

In [ ]:
print("Q_step, slope, subband-component number")
for _i, _j in enumerate(filtered_slopes):
    print(_i, "---", _j)

### Put all the lists together

In [ ]:
single_list = []
for l in filtered_slopes:
    for i in l:
        single_list.append(i)

In [ ]:
print("(quantization step, slope, subband-component number)")
single_list

### Sort the list of quantization steps, slopes, and subband-components
By slopes.

In [ ]:
progression_of_deltas = sorted(single_list, key=lambda _x: _x[1])[::-1]

In [ ]:
print("(quantization step, slope, subband-component number)")
progression_of_deltas

In [ ]:
print("Total number of subband-components =", (N_levels*3+1)*N_components)

### Let's see the progression of quantization steps

In [ ]:
Q_steps_pattern = 3*[''] # There are 3 color components
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            Q_steps_pattern.append('')
            
for i, j in enumerate(progression_of_deltas):
    Q_steps_pattern[j[2]] = j[0]
    print(i, j, Q_steps_pattern)

In [ ]:
def show(d, s):
    c = 0
    for i in d[0]:
        print(s, i, end=' ')
        c += 1
        if c > 3:
            break
    for sr in d[1:]:
        for sb in sr:
            c = 0
            for i in sb:
                print(s, i, end=' ')
                c += 1
                if c > 3:
                    break
    print()

def _quantize(decom, Q_steps):
    decom_k = []
    decom_k.append(Q.quantize_components(decom[0], Q_steps[0:3]))
    for sr in decom[1:]:
        decom_k.append(Q.sr)
    print()
    return decom_k

def _dequantize(decom_k, Q_steps):
    decom_dQ = []
    decom_dQ.append(decom_k[0])
    for sr_k in decom_k[1:]:
        decom_dQ.append(sr_k)
    print()
    return decom_dQ
    
def quantize_components_uint16(x, Q_steps):
    '''Quantize each component of <x> using a quantization step per component.'''
    x_k = np.empty_like(x, dtype=np.int32)
    for c in range(x.shape[2]):
        x_k[..., c] = Q.quantize(x[..., c], Q_steps[c]) + 32768
    return x_k.astype(np.uint16)

def dequantize_components_uint16(x_k, Q_steps):
    '''Dequantize each component of <x_k> using an independent quantization step.'''
    x_dQ = np.empty_like(x_k, dtype=np.int32)
    for c in range(x_k.shape[2]):
        x_dQ[..., c] = Q.dequantize(x_k[..., c].astype(np.int32) - 32768, Q_steps[c])
    return x_dQ

def quantize(decom, Q_steps):
    '''Quantize and add 32768.'''
    decom_k = []
    #decom_k.append(decom[0])
    decom_k.append(quantize_components_uint16(decom[0], Q_steps[0:3]))
    #print(decom[0].max(), decom[0].min(), decom_k[0].max(), decom_k[0].min())
    sbc_number = 3
    for sr in decom[1:]:
        sr_k = []
        for sb in sr:
            sb_k = quantize_components_uint16(sb, Q_steps[sbc_number:sbc_number + 3])
            sr_k.append(sb_k)
            sbc_number += 3
        #decom_k.append(sr)
        decom_k.append(tuple(sr_k))
    return decom_k

def dequantize(decom_k, Q_steps):
    decom_dQ = []
    #decom_dQ.append(decom_k[0])
    decom_dQ.append(dequantize_components_uint16(decom_k[0], Q_steps[0:3]))
    sbc_number = 3
    for sr_k in decom_k[1:]:
        sr_dQ = []
        for sb_k in sr_k:
            sb_dQ = dequantize_components_uint16(sb_k, Q_steps[sbc_number:sbc_number + 3])
            sr_dQ.append(sb_dQ)
            sbc_number += 3
        #decom_dQ.append(sr_k)
        decom_dQ.append(tuple(sr_dQ))
    return decom_dQ
    
def quantize_sr(sr, Q_step):
    '''Quantize a spatial resolution of the DWT (3 multicomponent subbands) using
    3*N_components independent quantization steps'''
    return tuple([Q.quantize(i, Q_step) for i in sr])

def _quantize(decomposition, Q_steps):
    LL = decomposition[0]
    LL_k = np.empty_like(LL, dtype=np.int16)
    for _c in range(N_components):
        LLc = LL[..., _c]
        LL_k[..., _c] = LLc # Q.quantize(LLc, Q_steps[_c])
    decomposition_k = [LL_k]
    sbc_number = 3
    for sr in decomposition[1:]:
        #print("sr", len(sr))
        sr_k = []
        for sb in sr:
            sbc_k = np.empty_like(sb)
            for _c in range(N_components):
                sbc = sb[..., _c]
                #print(len(Q_steps), sbc_number)
                sbc_k[..., _c] = sbc # Q.quantize(sbc, Q_steps[sbc_number])
                #sbc_k = Q.quantize(sbc, Q_steps[sbc_number])
                sbc_number += 1
            sr_k.append(sbc_k)
        decomposition_k.append(tuple(sr_k))
    return decomposition
    #return decomposition_k

def _dequantize(decomposition_k, Q_steps):
    LL_k = decomposition_k[0]
    LL_dQ = np.empty_like(LL_k).astype(np.float64)
    for _c in range(N_components):
        LLc_k = LL_k[..., _c]
        LL_dQ[..., _c] = LLc_k# Q.dequantize(LLc_k, Q_steps[_c])
    decomposition_dQ = [LL_dQ]
    sbc_number = 3
    for sr_k in decomposition_k[1:]:
        sr_dQ = []
        #print("sr_k", len(sr_k))
        for sb_k in sr_k:
            sbc_dQ = np.empty_like(sb_k).astype(np.float64)
            for _c in range(N_components):
                sbc_k = sb_k[..., _c]
                #print(len(Q_steps), sbc_number)
                sbc_dQ[..., _c] = sbc_k #Q.dequantize(sbc_k, Q_steps[sbc_number])
                sr_dQ.append(sbc_dQ)
                sbc_number += 1
        decomposition_dQ.append(tuple(sr_dQ))
    return decomposition_k
    #return decomposition_dQ

def resolution_level(sb_number):
    '''Resolution level in decomposition.'''
    if sb_number > 0:
        return ((sb_number - 1) // 3) + 1
    else:
        return 0
    
def subband_index(sb_number):
    '''Subband index in resolution level.'''
    if sb_number > 0:
        return (sb_number % 3) - 1
    else:
        return 0

### Find the optimal RD curve

In [ ]:
DWT_RDO_16_glued = []

# Initially all the quantization steps structure are "infinite"=10^4.
Q_steps_pattern = 3*[10**4] # There are 3 color components
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            Q_steps_pattern.append(10**4)

slope_index = 0
for s in progression_of_deltas:
    sbc_number = s[2]
    Q_steps_pattern[sbc_number] = s[0]
    decom_k = quantize(decom, Q_steps_pattern)
    BPP = (DWT.write_glued(decom_k, f"/tmp/optimal_{slope_index}_", 0)[0]*8)/YUV_img.size
    decom_dQ = dequantize(decom_k, Q_steps_pattern)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = np.clip(YUV.to_RGB(YUV_img_dQ), a_min=0, a_max=255)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_steps_pattern={Q_steps_pattern} BPP={BPP} RMSE={RMSE}")
    #if not slope_index % 10:
    #    image_3.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")
    DWT_RDO_16_glued.append((BPP, RMSE))
    slope_index += 1

In [ ]:
DWT_RDO_16_glued

In [ ]:
DCT_RDO = []
with open(HOME + "/Sistemas-Multimedia.github.io/milestones/07-DCT/DCT_RDO.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        DCT_RDO.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_8b_constantQ), label=f"{YUV.name}/{N_levels}-{wavelet_name} (glued, 8 bits, without RDO)")
pylab.plot(*zip(*DWT_RDO_16_glued), label=f"{YUV.name}/{N_levels}-{wavelet_name} (glued, 16 bits, with RDO)")
pylab.plot(*zip(*DCT_RDO), label="YCoCg/8x8-DCT (with RDO)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc="best")
pylab.show()

## Using unglued PNG images

### Read the image and move it to the 0-mean YCoCg domain.

In [ ]:
img = image_3.read(test_image, 0)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])

### Compute (the 8-bit) 2D-DWT

In [ ]:
decom = DWT.analyze(YUV_img, wavelet, N_levels)

### Create a list of RD points for each RD curve (subband-component) and the corresponding lists of RD slopes between these points

In [ ]:
RD_points = []
RD_slopes = []
for _c in range(N_components):
    sbc = decom[0][..., _c]#.astype(np.int32) << N_levels #* math.pow(2, N_levels)
    RD_points.append([(0, information.average_energy(sbc))]) # Work with RMSE's that are average distortions
    RD_slopes.append([])
level = N_levels - 1
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            sbc = sb[..., _c]#.astype(np.int32) << level #* math.pow(2, level)
            sbc_avg_energy = information.average_energy(sbc)
            # The first point of each RD curve has a maximum distortion equal
            # to the energy of the subband and a rate = 0
            RD_points.append([(0, sbc_avg_energy)])
            RD_slopes.append([])
    level -= 1

In [ ]:
print("subband-component, RD-point")
for _i,_j in enumerate(RD_points):
    print(_i,_j)
    if not ((_i+1) % 3):
        print('')

#### Populate the rest of points of each curve

In [ ]:
# Subband LL
sbc_number = 0
for _c in range(N_components):
    sbc = decom[0][..., _c]#.astype(np.int32) << N_levels #* math.pow(2, N_levels)
    Q_step_number = 0
    for Q_step in Q_steps:
        #_Q_step = compute_Q_step(Q_step, N_levels)
        sbc_k = Q.quantize(sbc, Q_step)
        sbc_dQ = Q.dequantize(sbc_k, Q_step)
        RMSE = distortion.RMSE(sbc, sbc_dQ)
        #print(sbc.max(), sbc.min(), sbc_dQ.max(), sbc_dQ.min())
        #BPP = image_1.write((sbc_k + 128).astype(np.uint8), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
        BPP = image_1.write((sbc_k + 32768).astype(np.uint16), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
        #BPP = information.entropy(sbc_k.astype(np.int16).flatten())
        point = (BPP, RMSE)
        RD_points[sbc_number].append(point)
        delta_BPP = BPP - RD_points[_c][Q_step_number][0]
        delta_RMSE = RD_points[_c][Q_step_number][1] - RMSE
        if delta_BPP > 0:
            slope = delta_RMSE/delta_BPP
        else:
            slope = 0#100**10
        print(f"sbc_number={sbc_number} Q_step={Q_step} BPP={point[0]} RMSE={point[1]} slope={slope}")
        #RD_slopes[_c].append((Q_step, slope, _c))
        #RD_slopes[_c].append((_Q_step, slope, _c))
        if slope > 0:
            RD_slopes[_c].append((Q_step, slope, _c))
        Q_step_number += 1
    sbc_number += 1

# Rest of subbands
level = N_levels - 1
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            sbc = sb[..., _c]#.astype(np.int32) << level #* math.pow(2, level)
            Q_step_number = 0
            for Q_step in Q_steps:
                #_Q_step = compute_Q_step(Q_step, level)
                sbc_k = Q.quantize(sbc, Q_step)
                sbc_dQ = Q.dequantize(sbc_k, Q_step)
                RMSE = distortion.RMSE(sbc, sbc_dQ)
                #BPP = image_1.write((sbc_k + 128).astype(np.uint8), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
                BPP = image_1.write((sbc_k + 32768).astype(np.uint16), f"/tmp/{sbc_number}_{Q_step}_", 0)*8/sbc.size
                #sbc_BPP = information.entropy(sbc_k.astype(np.int16).flatten())
                point = (BPP, RMSE)
                RD_points[sbc_number].append(point)
                delta_BPP = BPP - RD_points[sbc_number][Q_step_number][0]
                delta_RMSE = RD_points[sbc_number][Q_step_number][1] - RMSE
                if delta_BPP > 0:
                    slope = delta_RMSE/delta_BPP
                else:
                    slope = 0
                print(f"sbc_number={sbc_number} Q_step={Q_step} BPP={point[0]} RMSE={point[1]} slope={slope}")
                if slope > 0:
                    RD_slopes[sbc_number].append((Q_step, slope, sbc_number))
                Q_step_number += 1
            sbc_number += 1
    level -= 1

In [ ]:
print("(BPP, RMSE)")
for _i, _j in enumerate(RD_points):
    print(_i, _j)
    if not ((_i+1) % 3):
        print('')

In [ ]:
print("subband-component, (Q_step, slope, subband-component number)")
for _i, _j in enumerate(RD_slopes):
    print(_i, _j)
    if not ((_i+1) % 3):
        print('')

## Remove points that do not belong to the convex-hull 

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[1] < curr[1]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_slopes(i))

### Put all the lists together

In [ ]:
single_list = []
for l in filtered_slopes:
    for i in l:
        single_list.append(i)

In [ ]:
print("(quantization step, slope, subband-component number)")
single_list

### Sort the list of quantization steps, slopes, and subband-components
By slopes.

In [ ]:
progression_of_deltas = sorted(single_list, key=lambda _x: _x[1])[::-1]

In [ ]:
print("(quantization step, slope, subband-component number)")
progression_of_deltas

In [ ]:
print("Total number of subband-components =", (N_levels*3+1)*N_components)

### Let's see the progression of quantization steps

In [ ]:
Q_steps_pattern = 3*[''] # There are 3 color components
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            Q_steps_pattern.append('')
            
for i, j in enumerate(progression_of_deltas):
    Q_steps_pattern[j[2]] = j[0]
    print(i, j, Q_steps_pattern)

In [ ]:
def quantize_components(x, Q_steps):
    '''Quantize each component of <x> using a quantization step per component.'''
    x_k = np.empty_like(x, dtype=np.int32)
    for c in range(x.shape[2]):
        x_k[..., c] = Q.quantize(x[..., c], Q_steps[c]) # + 128
    return x_k

def dequantize_components(x_k, Q_steps):
    '''Dequantize each component of <x_k> using an independent quantization step.'''
    x_dQ = np.empty_like(x_k, dtype=np.int32)
    for c in range(x_k.shape[2]):
        #x_dQ[..., c] = Q.dequantize(x_k[..., c].astype(np.uint32) - 128, Q_steps[c])
        x_dQ[..., c] = Q.dequantize(x_k[..., c], Q_steps[c])
    return x_dQ

def quantize(decom, Q_steps):
    decom_k = []
    #decom_k.append(decom[0])
    decom_k.append(quantize_components(decom[0], Q_steps[0:3]))
    sbc_number = 3
    for sr in decom[1:]:
        sr_k = []
        for sb in sr:
            sb_k = quantize_components(sb, Q_steps[sbc_number:sbc_number + 3])
            sr_k.append(sb_k)
            sbc_number += 3
        #decom_k.append(sr)
        decom_k.append(tuple(sr_k))
    return decom_k

def dequantize(decom_k, Q_steps):
    decom_dQ = []
    #decom_dQ.append(decom_k[0])
    decom_dQ.append(dequantize_components(decom_k[0], Q_steps[0:3]))
    sbc_number = 3
    for sr_k in decom_k[1:]:
        sr_dQ = []
        for sb_k in sr_k:
            sb_dQ = dequantize_components(sb_k, Q_steps[sbc_number:sbc_number + 3])
            sr_dQ.append(sb_dQ)
            sbc_number += 3
        #decom_dQ.append(sr_k)
        decom_dQ.append(tuple(sr_dQ))
    return decom_dQ

### Find the optimal RD curve

In [ ]:
DWT_RDO_16_unglued = []

# Initially all the quantization steps structure are "infinite"=10^4.
Q_steps_pattern = 3*[10**4] # There are 3 color components
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            Q_steps_pattern.append(10**4)

slope_index = 0
for s in progression_of_deltas:
    sbc_number = s[2]
    Q_steps_pattern[sbc_number] = s[0]
    decom_k = quantize(decom, Q_steps_pattern)
    disk_decom_k = [add_offset(decom_k[0])]
    for sr_k in decom_k[1:]:
        disk_sr_k = []
        for sb_k in sr_k:
            disk_sr_k.append(add_offset(sb_k))
        disk_decom_k.append(tuple(disk_sr_k))
    BPP = (DWT.write_unglued(disk_decom_k, f"/tmp/optimal_{slope_index}_", 0)[0]*8)/img.size
    #BPP = (write_compact_decomposition(decom_k, f"/tmp/optimal_{slope_index}_", 0)*8)/YUV_img.size
    #BPP = entropy(y_k)
    decom_dQ = dequantize(decom_k, Q_steps_pattern)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = np.clip(YUV.to_RGB(YUV_img_dQ), a_min=0, a_max=255)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_steps_pattern={Q_steps_pattern} BPP={BPP} RMSE={RMSE}")
    #if not slope_index % 10:
    #    image_3.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")
    DWT_RDO_16_unglued.append((BPP, RMSE))
    slope_index += 1

In [ ]:
DWT_RDO_16_unglued

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_8b_constantQ), label=f"{YUV.name}/{N_levels}-{wavelet_name} (glued, 8 bits, without RDO)")
pylab.plot(*zip(*DWT_RDO_16_glued), label=f"{YUV.name}/{N_levels}-{wavelet_name} (glued, 16 bits, with RDO)")
pylab.plot(*zip(*DWT_RDO_16_unglued), label=f"{YUV.name}/{N_levels}-{wavelet_name} (unglued, 16 bits, with RDO)")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc="best")
pylab.show()

## RDO using bi-orthogonal filters (TODO)

## JPEG2000 RD curve (and comparison) (TODO)

## Ignore the rest ...

In [ ]:
def __read_image(prefix):
    x = image_3.read(prefix, 0)
    if len(x.shape) == 2:
        extended_x = np.zeros(shape=(x.shape[0],  x.shape[1], 3), dtype=np.uint16) 
        extended_x[..., 0] = x
        return extended_x
    else:
        return x

def _write_compact_decomposition(decom, prefix, image_number):
    rows = decom[len(decom)-1][0].shape[0]*2
    cols = decom[len(decom)-1][0].shape[1]*2
    coms = decom[0].shape[2]
    image_shape = (rows, cols, coms)
    view = np.empty(image_shape, np.uint16)

    # LL subband
    view[0:decom[0].shape[0],
         0:decom[0].shape[1]] = (decom[0].astype(np.int32) + 32768).astype(np.uint16)

    for l in range(len(decom)-1):

        # LH
        view[0:decom[l+1][0].shape[0],
             decom[l+1][0].shape[1]:decom[l+1][0].shape[1]*2] =\
                (decom[l+1][0].astype(np.int32) + 32768).astype(np.uint16)

        # HL
        view[decom[l+1][1].shape[0]:decom[l+1][1].shape[0]*2,
             0:decom[l+1][1].shape[1]] =\
                (decom[l+1][1].astype(np.int32) + 32768).astype(np.uint16)

        # HH
        view[decom[l+1][2].shape[0]:decom[l+1][2].shape[0]*2,
             decom[l+1][2].shape[1]:decom[l+1][2].shape[1]*2] =\
                (decom[l+1][2].astype(np.int32) + 32768).astype(np.uint16)
            
    return image.write(view, prefix, image_3_number)
    
def write_compact_decomposition(decom, prefix, image_number):
    rows = decom[len(decom)-1][0].shape[0]*2
    cols = decom[len(decom)-1][0].shape[1]*2
    coms = decom[0].shape[2]
    image_shape = (rows, cols, coms)
    view = np.empty(image_shape, np.uint8)

    # LL subband
    view[0:decom[0].shape[0],
         0:decom[0].shape[1]] = (decom[0].astype(np.int16) + 128).astype(np.uint16)

    for l in range(len(decom)-1):

        # LH
        view[0:decom[l+1][0].shape[0],
             decom[l+1][0].shape[1]:decom[l+1][0].shape[1]*2] =\
                (decom[l+1][0].astype(np.int16) + 128).astype(np.uint16)

        # HL
        view[decom[l+1][1].shape[0]:decom[l+1][1].shape[0]*2,
             0:decom[l+1][1].shape[1]] =\
                (decom[l+1][1].astype(np.int16) + 128).astype(np.uint16)

        # HH
        view[decom[l+1][2].shape[0]:decom[l+1][2].shape[0]*2,
             decom[l+1][2].shape[1]:decom[l+1][2].shape[1]*2] =\
                (decom[l+1][2].astype(np.int16) + 128).astype(np.uint16)
            
    return image_3.write(view, prefix, image_number)

def read_compact_decomposition(prefix, image_number, N_levels):
    view = image.read(prefix, image_number)
    wavelet = pywt.Wavelet("Haar")
    decom = DWT.analyze(np.zeros_like(view), wavelet, N_levels)
    
    # LL subband
    decom[0][...] = view[0:decom[0].shape[0],
                         0:decom[0].shape[1]] - 32768
    
    for l in range(len(N_levels)):
        
        # LH
        decom[l+1][0] =\
            view[0:decom[l+1][0].shape[0],
                 decom[l+1][0].shape[1]:decom[l+1][0].shape[1]*2] - 32668
            
        # HL
        decom[l+1][1] =\
            view[decom[l+1][1].shape[0]:decom[l+1][1].shape[0]*2,
                 0:decom[l+1][1].shape[1]] - 32768
            
        # HH
        decom[l+1][2] =\
            view[decom[l+1][2].shape[0]:decom[l+1][2].shape[0]*2,
                 decom[l+1][2].shape[1]:decom[l+1][2].shape[1]*2] - 32768

    return decom



In [ ]:
# No me borres todvía (mira lo que tengo dentro)

img = image_3.read(test_image)
YUV_img = YUV.from_RGB(img.astype(np.int16))
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
DWT_RDO = []
# Initially all the quantization steps structure are "infinite"=10^4.
Q_steps_pattern = 3*[10**4] # There are 3 color components
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            Q_steps_pattern.append(10**4)
for s in progression_of_deltas:
    decom_8b = to_8bits(decom)
    LL_8b = decom_8b[0]
    _Q_step = compute_Q_step(Q_step, N_levels)
    LL_8b_k = Q.quantize(LL_8b, _Q_step) # Baybe bettter Q.get_indexes()
    decom_8b_k = [(LL_8b_k + 128).astype(np.uint8)]
    LL_8b_dQ = Q.dequantize(LL_8b_k, _Q_step) # Q.get_signal()
    decom_8b_dQ = [LL_8b_dQ]
    sbc_counter = 0
    level = N_levels - 1
    for sr_8b in decom_8b[1:]: # sr = spatial_resolution
        sr_8b_k = []
        sr_8b_dQ = []
        for sb_8b in sr_8b: # sb = subband
            _Q_step = compute_Q_step(Q_step, level)
            sb_8b_k = Q.quantize(sb_8b, _Q_step)
            sb_8b_dQ = Q.dequantize(sb_8b_k, _Q_step)
            sr_8b_k.append((sb_8b_k + 128).astype(np.uint8))
            sr_8b_dQ.append(sb_8b_dQ)
            sbc_counter += 1
        decom_8b_k.append(tuple(sr_8b_k))
        decom_8b_dQ.append(tuple(sr_8b_dQ))
        level -= 1
    BPP = (DWT.write_glued(decom_8b_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/YUV_img.size
    decom_dQ = to_16bits(decom_8b_dQ)
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)
    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"Q_step={Q_step} BPP={BPP} RMSE={RMSE}")
    DWT_RDO.append((BPP, RMSE))
    #image_3.show(img_dQ.astype(np.uint8), f"Q_step={Q_step} BPP={BPP:3.2f} RMSE={RMSE:4.2f}")

In [ ]:
def ___estimate_codestream_len(x, prefix):
    empty_x = np.zeros_like(x)
    real_BPP = image_1.write(x.astype(np.uint8), prefix)
    empty_BPP = image_1.write(empty_x.astype(np.uint8), prefix)
    return real_BPP - empty_BPP
    #return information.entropy(x.astype(np.int16).flatten())*x.size

### Remove 0-slopes
In each subband-component, remove those zero contributions to the quality.

In [ ]:
def filter_0_slopes(slopes):
    filtered_slopes = []
    for i in slopes:
        if i[1]!= 0:
            filtered_slopes.append(i)
        else:
            print(f"removed {i}")
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_0_slopes(i))

## Remove points that do not belong to the convex-hull
Remove those points of each subband-component that do not satisfy that the slope is higher than the next point of the curve. 

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[1] < curr[1]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_slopes(i))

In [ ]:
print("Q_step, slope, subband-component number")
for _i, _j in enumerate(filtered_slopes):
    print(_i, "---", _j)

In [ ]:
# Show the slopes
RD_slopes_without_sbc_index = []
#RD_slopes_without_sbc_index.append([])
for _c in range(N_components):
    RD_slopes_without_sbc_index.append([])
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            RD_slopes_without_sbc_index.append([])
for Q_step in range(len(Q_steps)):
    RD_slopes_without_sbc_index[0].append(RD_slopes[0][Q_step][0:2])
sbc_number = 3
for sr in decom[1:]:
    for sb in sr:
        for _c in range(N_components):
            for Q_step in range(len(Q_steps)):
                RD_slopes_without_sbc_index[sbc_number].append(RD_slopes[sbc_number][Q_step][0:2])
            sbc_number += 1
print(RD_slopes_without_sbc_index[0])
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_slopes_without_sbc_index[0]), label="0", marker=0)
for sbc_number in range(10):
    pylab.plot(*zip(*RD_slopes_without_sbc_index[sbc_number]), label=f"{sbc_number}", marker=sbc_number)
pylab.title("Slopes of the RD curves of the subbands")
pylab.xlabel("Q_step")
pylab.ylabel("Slope")
plt.legend(loc="best")
pylab.show()

In [ ]:
def compute_Q_step(Q_step, level):
    #step = int(math.ceil(Q_step * math.pow(2, N_levels)))
    step = int(math.ceil(Q_step * math.pow(2, level)))
    print(f"compute_Q_step: Q_step={Q_step} level={level} step={step}")
    return step

In [ ]:
Q_steps = [16, 8, 4, 2, 1, 0.5, 0.25, 1/8, 1/16]
# Use "uniform" quantization and write_unglued()
#img = image_3.read(test_image)
#YUV_img = YUV.from_RGB(img.astype(np.int16) - 128)

img = image_3.read(test_image).astype(np.int16)
YUV_img = YUV.from_RGB(img)
avgs = [np.average(YUV_img[..., c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)

image_domain_DWT_RD_points2 = []
for Q_step in Q_steps:
    LL = decom[0]
    print('LL')
    _Q_step = compute_Q_step(Q_step, N_levels)
    #LL_k = Q.quantize(LL, Q_step << N_levels) # Baybe bettter Q.get_indexes()
    LL_k = Q.quantize(LL, _Q_step) # Baybe bettter Q.get_indexes()
    #LL_k = Q.quantize(LL, Q_step) # Baybe bettter Q.get_indexes()
    #assert (-128 <= LL_k).all() and (LL_k <= 127).all(), print("LL", Q_step, np.unique(LL_k))
    #LL_dQ = Q.dequantize(LL_k, Q_step << N_levels) # Q.get_signal()
    LL_dQ = Q.dequantize(LL_k, _Q_step) # Q.get_signal()
    #LL_dQ = Q.dequantize(LL_k, Q_step) # Q.get_signal()
    if drange(LL_k) < 256:
        decom_k = [(LL_k + 128).astype(np.uint8)]
    else:
        decom_k = [(LL_k + 32768).astype(np.uint16)]
    decom_dQ = [LL_dQ]
    #dist = distortion.RMSE(LL, LL_dQ)
    #RMSE = (dist * LL.size)/x.size
    #print(gains[0], dist, gains[0] * dist, RMSE)
    #for i in range(4):
    #    for j in range(4):
    #        print(LL[i, j], LL_dQ[i, j])
    sbc_counter = 0
    levels_counter = N_levels - 1
    for sr in decom[1:]: # sr = spatial_resolution
        sr_k = []
        sr_dQ = []
        for sb in sr: # sb = subband
            #print(RMSE)
            #sb_k = Q.quantize(sb, Q_step << levels_counter)
            print(f"levels_counter={levels_counter}")
            _Q_step = compute_Q_step(Q_step, levels_counter)
            sb_k = Q.quantize(sb, _Q_step)
            #sb_k = Q.quantize(sb, Q_step)
            #assert (-128 <= sb_k).all() and (sb_k <= 127).all(), print(sbc_counter, np.unique(sb_k))
            #sb_dQ = Q.dequantize(sb_k, Q_step << levels_counter)
            sb_dQ = Q.dequantize(sb_k, _Q_step)
            #sb_dQ = Q.dequantize(sb_k, Q_step)
            if drange(sb_k) < 256:
                sr_k.append((sb_k + 128).astype(np.uint8))
            else:
                sr_k.append((sb_k + 32768).astype(np.uint16))
            sr_dQ.append(sb_dQ)
            #dist = distortion.RMSE(sb, sb_dQ)
            #print(gains[counter], dist, gains[counter] * dist, RMSE)
            #MSE += (dist * sb.size)/x.size
            sbc_counter += 1
        decom_k.append(tuple(sr_k))
        decom_dQ.append(tuple(sr_dQ))
        levels_counter += 1
    #subbands_info(decom_dQ)
    #BPP = (write_compact_decomposition(decom_k, f"/tmp/constant_{Q_step}_", 0)*8)/YUV_img.size
    #shifted_decom_k = DWT.add(decom_k, 128)
    #shifted_decom_k = DWT.set_type(decom_k, np.uint8)
    #BPP = (DWT.write(decom_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/YUV_img.size
    BPP = (DWT.write_unglued(decom_k, f"/tmp/constant_{Q_step}_", 0)[0]*8)/YUV_img.size
    YUV_img_dQ = DWT.synthesize(decom_dQ, wavelet, N_levels)

    for c in range(3):
        YUV_img_dQ[..., c] += int(avgs[c])
    img_dQ = YUV.to_RGB(YUV_img_dQ)
    img_dQ = np.clip(img_dQ, a_min=0, a_max=255).astype(np.uint8)

    #img_dQ = np.clip(YUV.to_RGB(YUV_img_dQ), a_min=-128, a_max=127) + 128
    #img_dQ = YUV.to_RGB(YUV_img_dQ) + 128
    #clipped_img_dQ = np.clip(img_dQ, a_min=0, a_max=255)
    #RMSE = distortion.RMSE(img, clipped_img_dQ)
    RMSE = distortion.RMSE(img, img_dQ)
    print(f"_Q_step={_Q_step} BPP={BPP} RMSE={RMSE}")
    image_domain_DWT_RD_points2.append((BPP, RMSE))
    image_3.show(img_dQ.astype(np.uint8), f"Reconstructed image (Q_step={Q_step})")

## Lossy compression

In [ ]:
img = image_3.read(test_image).astype(np.int16)
YUV_img = YUV.from_RGB(img)
avgs = [np.average(YUV_img[...,c]) for c in range(3)]
print(f"avgs={avgs}")
for c in range(3):
    YUV_img[..., c] -= int(avgs[c])
decom = DWT.analyze(YUV_img, wavelet, N_levels)
decom8 = _8bits_me(decom)
decom8_ = DWT.add(decom8, 128)
decom8__ = DWT.set_type(decom8_, np.uint8)
output_len, slices = DWT.write_glued(decom8__, "/tmp/lossless", 0)
_decom8 = DWT.read_glued(slices, "/tmp/lossless", 0)
_decom8 = inverse_8bits_me(_decom8)
_decom8 = DWT.add(_decom8, -128)
_YUV_img8 = DWT.synthesize(_decom8, wavelet, N_levels)
for c in range(3):
    _YUV_img[..., c] += int(avgs[c])
_img = YUV.to_RGB(_YUV_img).astype(np.uint8)
image_3.show(_img)

In [ ]:
image_3.show(img - _img + 128)

In [ ]:
image_3.show(image_3.normalize(img - _img + 128))

In [ ]:
subbands_info(decom8)

### Quantization steps

The high dynamic range of the wavelet coefficients dificults the use 8-bit RGB PNG encoding, which is more compact that the 16-bit version. At this point, we have basically two alternatives:
1. Use a set of high enough quantization steps to keep the quantization indexes into 8 bits. However, notice that this probably is going to ignore most of the information of the high-frequency subbands because the amplitude of their coefficients are smaller than the smaller quantization step.
2. If the quality provided by the smaller quantization step is not enough, the unencoded (least significant) bit-planes of the coefficients can be compressed in a second 8-bit PNG image. TO-DO.
3. Check the dynamic range of the quantization idexes and if is larger than [-128, 127] use 16 bpps.

Notice that the smaller quantization step dependens on several factors:
1. The wavelet.
2. The number of levels.
3. The image content.

In [ ]:
#Q_steps = [4096, 2048, 1024, 512, 256, 128]
#Q_steps = [256, 128, 64, 32, 16]
#Q_steps = [64, 32, 16]
Q_steps = [32, 16, 8, 4, 2, 1, 0.5]
#Q_steps = [2]

## Uniform quantization vs optimal quantization

As we did with the DCT, let's compare both types of quantization in the RD space. Steps:

1. Compute the RD slope of each subband for a set of quantization steps. For this we will suppose that the subbands are independent (the DWT is orthogonal), measuring the distortion in the wavelet domain.
2. Sort the slopes. This will determine the optimal progression of quantization steps (which subband to incorporate more data to the code-stream, progressively).
3. Compute the distortion in the image domain for each bit-rate. Notice that this information should match with the privided by the step 1 (measuring the distortion in the wavelet domain). However, we prefer to computate the distortion in the image domain because the transform does not need to be orthogonal.

 Read the image, move to the YUV domain, and compute the DWT.

In [ ]:
xx = read_image(test_image)
x = YUV.from_RGB(xx.astype(np.int16) - 128)
y = DWT.analyze(x, wavelet, N_levels)

For each subband, we populate:
1. A list of RD points, and
2. A list of RD slopes with these points, indicanting also the corresponding quantization step and subband.

Remember that we have a RD point for each quantization step for each subband. The first dimension of these lists is indexed the subband, and the second dimension is indexed by the quantization step.

In [ ]:
# For BPP=0, the RMSE is the energy of the subband. No slope can be computed for the first point.
RD_points = [[(0, information.energy(y[0]) / y[0].size)]] # Work with RMSE's that are average distortions
RD_slopes = [[]]
for sr in y[1:]:
    for sb in sr:
        sb_avg_energy = information.energy(sb) / sb.size
        # The first point of each RD curve has a maximum distortion equal
        # to the energy of the subband and a rate = 0
        RD_points.append([(0, sb_avg_energy)])
        RD_slopes.append([])

for i,j in enumerate(RD_points):
    print(i,j)
    
for i,j in enumerate(RD_slopes):
    print(i,j)

In [ ]:
# Now populate the rest of points of each subband

# Subband LL
sb_number = 0
sb = y[0]
Q_step_number = 0
for Q_step in Q_steps:
    print(Q_steps)
    sb_k = Q.quantize(sb, Q_step)
    sb_dQ = Q.dequantize(sb_k, Q_step)
    sb_RMSE = distortion.RMSE(sb, sb_dQ)
    sb_BPP = information.PNG_BPP((sb_k.astype(np.int32) + 32768).astype(np.uint16), "/tmp/BPP_")[0]
    #sb_BPP = information.entropy(sb_k.astype(np.int16).flatten())
    RD_points[sb_number].append((sb_BPP, sb_RMSE))
    delta_BPP = sb_BPP - RD_points[sb_number][Q_step_number][0]
    delta_RMSE = RD_points[sb_number][Q_step_number][1] - sb_RMSE
    if delta_BPP > 0:
        slope = delta_RMSE/delta_BPP
    else:
        slope = 0
    RD_slopes[sb_number].append((Q_step, slope, (sb_number)))
    Q_step_number += 1

print(N_levels)
    
for i,j in enumerate(RD_points):
    print(i, "---", j)
    
for i,j in enumerate(RD_slopes):
    print(i, "---", j)

In [ ]:
sb_number = 1
for sr in y[1:]:
    for sb in sr:
        Q_step_number = 0
        for Q_step in Q_steps:
            sb_k = Q.quantize(sb, Q_step)
            sb_dQ = Q.dequantize(sb_k, Q_step)
            sb_RMSE = distortion.RMSE(sb, sb_dQ)
            sb_BPP = information.PNG_BPP((sb_k.astype(np.int32) + 32768).astype(np.uint16), "/tmp/BPP_")[0]
            #sb_BPP = information.entropy(sb_k.astype(np.int16).flatten())
            RD_points[sb_number].append((sb_BPP, sb_RMSE))
            delta_BPP = sb_BPP - RD_points[sb_number][Q_step_number][0]
            delta_RMSE = RD_points[sb_number][Q_step_number][1] - sb_RMSE
            if delta_BPP > 0:
                slope = delta_RMSE/delta_BPP
            else:
                slope = 9^9
            print(sb_number, len(y))
            RD_slopes[sb_number].append((Q_step, slope, (sb_number)))
            Q_step_number += 1
        sb_number += 1
        
for i,j in enumerate(RD_points):
    print(i, "---", j)
    
for i,j in enumerate(RD_slopes):
    print(i, "---", j)

In [ ]:
RD_slopes

In [ ]:
if sb_number < 12:
    pylab.figure(dpi=150)
    pylab.plot(*zip(*RD_points[0]), label="0", marker=0)
    sb_number = 1
    for sr in y[1:]:
        for sb in sr:
            pylab.plot(*zip(*RD_points[sb_number]), label=f"{sb_number}", marker=sb_number)
            sb_number += 1
    pylab.title("RD curves of the subbands")
    pylab.xlabel("Bits/Pixel")
    pylab.ylabel("RMSE")
    plt.legend(loc="best")
    pylab.show()

In [ ]:
if sb_number < 12:
    pylab.figure(dpi=150)
    pylab.plot(*zip(*RD_points[0]), label="0", marker=0)
    sb_number = 1
    for sr in y[1:]:
        for sb in sr:
            pylab.plot(*zip(*RD_points[sb_number]), label=f"{sb_number}", marker=sb_number)
            sb_number += 1
    pylab.title("RD curves of the subbands")
    pylab.xlabel("Bits/Pixel")
    pylab.ylabel("RMSE")
    pylab.yscale("log")
    plt.legend(loc="best")
    pylab.show()

In [ ]:
RD_slopes_without_sb_index = []
RD_slopes_without_sb_index.append([])
for sr in y[1:]:
    for sb in sr:
        RD_slopes_without_sb_index.append([])
for Q_step in range(len(Q_steps)):
    RD_slopes_without_sb_index[0].append(RD_slopes[0][Q_step][0:2])
sb_number = 1
for sr in y[1:]:
    for sb in sr:
        for Q_step in range(len(Q_steps)):
            RD_slopes_without_sb_index[sb_number].append(RD_slopes[sb_number][Q_step][0:2])
        sb_number += 1
print(RD_slopes_without_sb_index[0])
if sb_number < 12:
    pylab.figure(dpi=150)
    pylab.plot(*zip(*RD_slopes_without_sb_index[0]), label="0", marker=0)
    sb_number = 1
    for sr in y[1:]:
        for sb in sr:
            pylab.plot(*zip(*RD_slopes_without_sb_index[sb_number]), label=f"{sb_number}", marker=sb_number)
            sb_number += 1
    pylab.title("Slopes of the RD curves of the subbands")
    pylab.xlabel("Q_step")
    pylab.ylabel("Slope")
    plt.legend(loc="best")
    pylab.show()

In [ ]:
if sb_number < 12:
    pylab.figure(dpi=150)
    pylab.plot(*zip(*RD_slopes_without_sb_index[0]), label="0", marker=0)
    sb_number = 1
    for sr in y[1:]:
        for sb in sr:
            pylab.plot(*zip(*RD_slopes_without_sb_index[sb_number]), label=f"{sb_number}", marker=sb_number)
            sb_number += 1
    pylab.title("Slopes of the RD curves of the subbands")
    pylab.xlabel("Q_step")
    pylab.ylabel("Slope")
    pylab.yscale("log")
    plt.legend(loc="best")
    pylab.show()

It can be seen that the slopes of the curves are quite similar, but the LL subband is somewhat steeper.

Let's sort the slopes.

In [ ]:
single_list = []
for Q_step in range(len(Q_steps)):
    single_list.append(tuple(RD_slopes[0][Q_step]))
sb_number = 1
for sr in y[1:]:
    for sb in sr:
        for Q_step in range(len(Q_steps)):
            single_list.append(tuple(RD_slopes[sb_number][Q_step]))
        sb_number += 1
sorted_slopes = sorted(single_list, key=lambda x: x[1])[::-1]

In [ ]:
sorted_slopes

In [ ]:
def quantize(decomposition, Q_steps):
    #print(Q_steps)
    LL = decomposition[0]
    LL_k = Q.quantize(LL, Q_steps[0])
    decomposition_k = [LL_k]
    sb_number = 1
    for sr in decomposition[1:]:
        sr_k = []
        for sb in sr:
            #print(sb_number)
            sb_k = Q.quantize(sb, Q_steps[sb_number])
            sr_k.append(sb_k)
            sb_number += 1
        decomposition_k.append(tuple(sr_k))
    return decomposition_k

def dequantize(decomposition_k, Q_steps):
    LL_k = decomposition_k[0]
    LL_dQ = Q.dequantize(LL_k, Q_steps[0])
    decomposition_dQ = [LL_dQ]
    sb_number = 1
    for sr_k in decomposition_k[1:]:
        sr_dQ = []
        for sb_k in sr_k:
            sb_dQ = Q.dequantize(sb_k, Q_steps[sb_number])
            sr_dQ.append(sb_dQ)
            sb_number += 1
        decomposition_dQ.append(tuple(sr_dQ))
    return decomposition_dQ

def resolution_level(sb_number):
    '''Resolution level in decomposition.'''
    if sb_number > 0:
        return ((sb_number - 1) // 3) + 1
    else:
        return 0
    
def subband_index(sb_number):
    '''Subband index in resolution level.'''
    if sb_number > 0:
        return (sb_number % 3) - 1
    else:
        return 0

In [ ]:
optimal_RD_points = []
#y_prog = DWT.analyze(np.zeros_like(x), wavelet, N_levels)
#print(len(y_prog))
Q_steps_by_subband = [9**9]
#for sr in y_prog[1:]:
for sr in y[1:]:
    #print(len(sr))
    for sb in sr:
        Q_steps_by_subband.append(9**9)
#print(Q_steps_by_subband)
slope_index = 0
for s in sorted_slopes:
    sb_number = s[2]
    #print("sb_number", sb_number)
    Q_steps_by_subband[sb_number] = s[0]
    #print(sb_number, Q_steps_by_subband[sb_number])
    #y_prog[resolution_level(sb_number)][subband_index(sb_number)] = y[resolution_level(sb_number)][subband_index(sb_number)]
    #y_k = quantize(y_prog, Q_steps_by_subband)
    y_k = quantize(y, Q_steps_by_subband)
    BPP = (write_compact_decomposition(y_k, f"/tmp/optimal_{slope_index}_", 0)*8)/xx.size
    #BPP = entropy(y_k)
    slope_index += 1
    y_dQ = dequantize(y_k, Q_steps_by_subband)
    z_dQ = DWT.synthesize(y_dQ, wavelet, N_levels)
    zz_dQ = np.clip(YUV.to_RGB(z_dQ), a_min=-128, a_max=127) + 128
    RMSE = distortion.RMSE(xx, zz_dQ)
    print(f"{Q_steps_by_subband} {BPP} {RMSE}")
    optimal_RD_points.append((BPP, RMSE))
    #image_3.show_RGB_image(zz_dQ, f"Reconstructed image (Q_step={Q_step})")

In [ ]:
optimal_RD_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_points), label="Uniform quantization")
pylab.plot(*zip(*optimal_RD_points), label="Optimal quantization")
#pylab.plot(*zip(*JPEG_RD_points), label="JPEG")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc="best")
pylab.show()

# .......

ESTE PASO NO ES NECESARIO: Compute the average energy of the image and the decomposition.

In [ ]:
def energy(decomposition):
    accumulated_energy = information.energy(decomposition[0])
    for sr in y[1:]:
        for sb in sr:
            accumulated_energy += information.energy(sb)
    return accumulated_energy

In [ ]:
xx = read_image(test_image).astype(np.int16) - 128
#xx = np.full(shape=(512, 512, 3), fill_value=100) - 128
x = YUV.from_RGB(xx)
y = DWT.analyze(x, wavelet, N_levels)
image_energy = information.average_energy(x)
print(image_energy)
print(energy(y)/x.size)

## Transform gains

This information measures whether the transform amplifies or attenuates the signal. If the forward transform amplifies the signal, the energy of the decomposition will be larger than the energy of the original signal, and viceversa. The same idea can be applied to the inverse transform.

In [ ]:
x = np.full(shape=(512, 512, 3), fill_value=1)
x_energy = information.energy(x)
y = DWT.analyze(x, wavelet, N_levels)
decom_energy = information.energy(y[0])
z = DWT.synthesize(y, wavelet, N_levels)
for sr in y[1:]:
    for sb in sr:
        decom_energy += information.energy(sb)
z_energy = information.energy(z)
print(wavelet)
print("Energy of the original image:", x_energy)
print("Energy of the decomposition:", decom_energy)
print("Energy of the reconstucted image:", z_energy)
print("Average energy of the original image", x_energy / x.size)
print("Average energy of the decomposition:", decom_energy / x.size)
print("Average energy of the reconstructed image:", z_energy / x.size)

As it can be seen, the transform is energy preserving, which means that we the distortion generated by quantization is the same in the image and the wavelet domains.

## RD performance considering (and not) the subband gains

We compute the RD cuve of using scalar quantization when:
1. All subbands are quantized using the same quantization step.
2. The quantization step used in a subband is divided by the subband gain.

In [ ]:
xx = read_image(test_image).astype(np.int16) - 128
x = YUV.from_RGB(xx)

constant_Q_points = []
for Q_step in Q_steps:
    y = DWT.analyze(x, wavelet, N_levels)
    LL = y[0]
    LL_k = Q.quantize(LL, Q_step)
    y_k = [LL_k]
    LL_dQ = Q.dequantize(LL_k, Q_step)
    y_dQ = [LL_dQ]
    for sr in y[1:]:
        sr_k = []
        sr_dQ = []
        for sb in sr:
            sb_k = Q.quantize(sb, Q_step)
            sr_k.append(sb_k)
            sb_dQ = Q.dequantize(sb_k, Q_step)
            sr_dQ.append(sb_dQ)
        y_k.append(tuple(sr_k))
        y_dQ.append(tuple(sr_dQ))
    BPP = (write_compact_decomposition(y_k, f"/tmp/constant_{Q_step}", 0)*8)/x.size
    z_dQ = DWT.synthesize(y_dQ, wavelet, N_levels)
    zz_dQ = np.clip(YUV.to_RGB(z_dQ), a_min=-128, a_max=127)
    MSE = distortion.MSE(xx, zz_dQ)
    print(f"{Q_step} {BPP} {MSE}")
    constant_Q_points.append((BPP, MSE))
    #image_3.show_RGB_image(zz_dQ + 128, f"Reconstructed image (Q_step={Q_step})")

Let's suppose that the slope of the subband is proportional to the subband gain.

In [ ]:
xx = read_image(test_image).astype(np.int16) - 128
x = YUV.from_RGB(xx)

relative_gains = [gain/gains[-1] for gain in gains]
print(relative_gains)
gains_Q_points = []
for Q_step in Q_steps:
    y = DWT.analyze(x, wavelet, N_levels)[::-1]
    counter = len(y) - 1
    for sr in y[:-1]:
        sr_k = []
        sr_dQ = []
        for sb in sr:
            _Q_step = Q_step / relative_gains[counter]
            print("Q_step =",_Q_step)
            sb_k = Q.quantize(sb, _Q_step)
            sr_k.append(sb_k)
            sb_dQ = Q.dequantize(sb_k, _Q_step)
            sr_dQ.append(sb_dQ)
            counter -= 1
        y_k.append(tuple(sr_k))
        y_dQ.append(tuple(sr_dQ))
    LL = y[-1]
    _Q_step = Q_step / relative_gains[0]
    print(_Q_step)
    LL_k = Q.quantize(LL, _Q_step)
    y_k = [LL_k]
    LL_dQ = Q.dequantize(LL_k, _Q_step)
    y_dQ = [LL_dQ]
    BPP = (write_compact_decomposition(y_k, f"/tmp/gains_{Q_step}", 0)*8)/x.size
    z_dQ = DWT.synthesize(y_dQ, wavelet, N_levels)
    zz_dQ = np.clip(YUV.to_RGB(z_dQ), a_min=-128, a_max=127)
    MSE = distortion.MSE(xx, zz_dQ)
    print(f"{Q_step} {BPP} {MSE}")
    gains_Q_points.append((BPP, MSE))
    image_3.show_RGB_image(zz_dQ + 128, f"Reconstructed image (Q_step={Q_step})")

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*constant_Q_points), label=f"Constant Q")
pylab.plot(*zip(*gains_Q_points), label=f"Gains Q")
pylab.title(test_image)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

## Optimal quantization progression

The previous quantization is not (usually) optimal, because the RD constribution of each subband is not constant.
Let's use now a different quantization step for each subband that operates (approximately) at the same RD slope.

## An example of uniform quantization

We will measure also the distortion in both domains.

In [ ]:
Q_step = 128
x = read_image(test_image).astype(np.int16) #- 128
#x = YUV.from_RGB(xx)
y = DWT.analyze(x, wavelet, N_levels)

LL = y[0]
LL_k = Q.quantize(LL, Q_step)
LL_dQ = Q.dequantize(LL_k, Q_step)
dist = distortion.MSE(LL, LL_dQ)
subband_ratio = LL.size / x.size
print(subband_ratio)
MSE_wavelet_domain = dist * subband_ratio #* gains[0]
counter = 1
y_dQ = [LL_dQ]
for sr in y[1:]:
    sr_dQ = []
    for sb in sr:
        sb_k = Q.quantize(sb, Q_step)
        sb_dQ = Q.dequantize(sb_k, Q_step)
        sr_dQ.append(sb_dQ)
        dist = distortion.MSE(sb, sb_dQ)
        subband_ratio = sb.size / x.size
        print(subband_ratio)
        MSE_wavelet_domain += dist * subband_ratio #* gains[counter]
        counter += 1
    y_dQ.append(tuple(sr_dQ))

z_dQ = DWT.synthesize(y_dQ, wavelet, N_levels)
cz_dQ = np.clip(z_dQ, a_min=0, a_max=255)
#zz_dQ = np.clip( YUV.to_RGB(z_dQ) + 128, a_min=0, a_max=255)
#zz_dQ = YUV.to_RGB(z_dQ) #+ 128
#print("Distortion in the image domain:", distortion.MSE(xx + 128, zz_dQ))
print("Distortion in the image domain:", distortion.MSE(x, cz_dQ))
print("Distortion in the wavelet domain:", MSE_wavelet_domain)
image_3.show_RGB_image_3(cz_dQ, f"Reconstructed image (Q_step={Q_step})")

## Optimal quantization progression

The previous quantization is not (usually) optimal, because the RD constribution of each subband is not constant.
Let's use now a different quantization step for each subband that operates (approximately) at the same RD slope.

## An orthogonality test

Orthogonality is necessary to avoid that the quantization error generated in a subband does not affect to the rest of subband. This will speed up the RD optimization because the distortion can be measured in the DWT domain.

This orthogonality test does:
1. Compute the DWT of an image.
2. Set to zero all the subbands except one.
3. Compute the inverse DWT.
4. Compute the DWT again of the previous reconstruction.
5. Test if the decomposition matches the one generated in the step 2.  If matches (with some maximum error), the transform is orthogonal.

In [ ]:
y = DWT.analyze(x, wavelet, N_levels)
subband_to_keep = 5
if subband_to_keep > DWT._N_levels:
    print("No way, José")
y[0][...] = 0.0
counter = 0
for sr in y[1:]:
    for sb in sr:
        if counter != subband_to_keep:
            sb[...] = 0.0
        counter += 1
z = DWT.synthesize(y, wavelet, N_levels)
#image_3.show_RGB_image(z, "Reconstructed image")
y2 = DWT.analyze(z, wavelet, N_levels)
counter = 0
orthogonal = True
for sr, sr2 in zip(y[1:], y2[1:]):
    for sb, sb2 in zip(sr, sr2):
        #print((sb == sb2).allclose())
        if not np.allclose(sb, sb2):
            orthogonal = False
        #if counter == subband_to_keep:
        #    image_3.show_RGB_image(sb)
        #    image_3.show_RGB_image(sb2)
        counter += 1
print("Orthogonal:", orthogonal)

Another way to know if the transform is orthogonal is compute the quantization distortion in the wavelet domain and see if it is the same than the distortion in the image domain. 

## Optimal quantization progression

The previous quantization is not (usually) optimal, because the RD constribution of each subband is not constant.
Let's use now a different quantization step for each subband that operates (approximately) at the same RD slope.

This information is important to known if the transform is unitary or not (usually, biorthogonal transforms are not unitary, i.e., the energy of the decomposition is different to (usually larger than) the energy of the image). Notice that if the transform is not unitary, the distortion is measured differently in the image and the transform domain. For example, is the gain is larger than 1, then overall distortion should be divided by the gain.

In [ ]:
x = read_image(test_image)
#x = YUV.from_RGB(xx)
y = DWT.analyze(x, wavelet, N_levels)
image_energy = distortion.energy(x)
image_average_energy = image_energy / x.size
print("Image average energy:", image_average_energy)
#decom_average_energy = distortion.average_energy(y[0])*y[0].size/x.size
decom_energy = distortion.energy(y[0])
counter = 1
for sr in y[1:]:
    for sb in sr:
        #decom_energy += distortion.average_energy(sb)*sb.size/x.size
        decom_energy += distortion.energy(sb)
        counter += 1
print("Decomposition energy", decom_energy)
decom_average_energy = decom_energy / x.size
print("Decomposition average energy", decom_average_energy)
forward_transform_gain = decom_energy/image_energy
print("Forward transform gain:", forward_transform_gain)
print("The transform is", end=' ')
try:
    np.testing.assert_almost_equal(forward_transform_gain, 1.0)
except AssertionError:
    print("not unitary")
else:
    print("unitary")

In [ ]:
# Read the image and move to the YCoCg domain.
x = image_3.read(test_image)
xx = YUV.from_RGB(x.astype(np.int16) - 128)
#xx = YUV.from_RGB(x.astype(np.int16))
yy = DWT.analyze(xx, wavelet, N_levels)
zz_dQ = DWT.synthesize(yy, wavelet, N_levels)
z_dQ = np.clip(YUV.to_RGB(zz_dQ) + 128, a_min=0, a_max=255).astype(np.uint8)
#z_dQ = YUV.to_RGB(zz_dQ).astype(np.uint8)
image_3.show(z_dQ)

## Testing `DWT.analyze_step()` and `DCT.synthesize_step()`

In [ ]:
x = image_3.read(test_image)
image_3.show(x, title="Original")

In [ ]:
L, H = DWT.analyze_step(x, wavelet)

In [ ]:
image_3.show(image_3.normalize(L), "LL DWT domain")
subbands = ("LH", "HL", "HH")
for i, sb in enumerate(subbands):
    image_3.show(image_3.normalize(H[i]), f"{sb} DWT domain")

In [ ]:
z = DWT.synthesize_step(L, H, wavelet).astype(np.uint8)

In [ ]:
r = x - z

In [ ]:
image_3.show(image_3.normalize(r), f"DWT finite precission error N_DWT_levels={N_levels}")

In [ ]:
r.max()

The DWT is not fully reversible, but it is almost.

In [ ]:
image_3.show(z, "Reconstructed image")

In [ ]:
class RGB_image_compression:
    pass

In [ ]:
class DWT_image_compression:
    def encode_RGB_image(img, Q_step):
        YUV_img = YUV.from_RGB(img.astype(np.int16))